In [ ]:
# **공고제목 - title**
# **회사이름 - company_name**
# **디테일 페이지로 가는 주소 - detail_url **
# **마감일 - end_date**
# ** 참고한 플랫폼 이름 - platform_name**
# ** 카테고리 - category_name**
# ** 기술 스택 - stack **
# ** 지역  - region **
# ** 신입/경력 - career **

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup



##### 마우스 이벤트 시작 #####

# 웹 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.jobplanet.co.kr/welcome/index/")
driver.implicitly_wait(5)

# 채용 버튼 클릭
button = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "job_posting"))
)
ActionChains(driver).click(button).perform()

# 직종 버튼 클릭
job_button = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), '직종')]"))
)
ActionChains(driver).click(job_button).perform()

# 데이터 버튼 클릭
data_button = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.XPATH, "//button[text()='데이터']"))
)
ActionChains(driver).click(data_button).perform()

# 데이터 전체 버튼 클릭
engineer_button = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.XPATH, "//span[text()='데이터 전체']"))
)
ActionChains(driver).click(engineer_button).perform()

# 적용 버튼 클릭
apply_button = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.XPATH, "//button[text()='적용']"))
)
ActionChains(driver).click(apply_button).perform()

##### 마우스 이벤트 종료 #####

##### 스크랩 시작 #####

# 페이지가 완전히 로드될 때까지 잠시 대기
time.sleep(10)

# 스크롤을 일정 횟수만큼 수행 (예: 5번)
scroll_pause_time = 1
scroll_limit = 5

for _ in range(scroll_limit):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# BeautifulSoup으로 페이지 파싱
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# 1. a 태그에서 href 속성을 추출
links = soup.find_all("a", "group z-0 block medium", title="페이지 이동")

# 2. 공고 정보 추출
job_elements = soup.find_all("div", "group mt-[16px] group-[.small]:mt-[14px] medium")

# 3. 결과를 저장할 리스트
jobs_data = []

# 4. 링크와 공고 정보를 함께 출력
for job, link in zip(job_elements, links):
    # 각 공고 정보를 저장할 딕셔너리
    job_data = {}
    
    # 제목 (h2 태그 사용)
    title = job.find("h2", "line-clamp-2 break-all text-h7 text-gray-800 group-[.small]:text-h8")
    if title:
        job_data["title"] = title.get_text(strip=True)

    # 회사명 (em 태그 사용)
    company_name = job.find("em", "inline-block w-full truncate text-body2 font-medium text-gray-800")
    if company_name:
        job_data["company_name"] = company_name.get_text(strip=True)

    # 기술 스택 (span 태그 사용)
    skills = job.find("span", "mt-[6px] inline-block w-full truncate text-small1 text-gray-500")
    if skills:
        job_data["skills"] = skills.get_text(strip=True)

    # 링크 (a 태그에서 href 속성 추출)
    job_data["detail_url"] = link.get("href")

    # 마감일 (임의로 상시 채용으로 설정)
    job_data["end_date"] = "상시 채용"

    # 플랫폼 이름
    job_data["platform_name"] = "Jobplanet"

    # 저장된 공고 데이터를 리스트에 추가
    jobs_data.append(job_data)

# 결과 출력 (예시)
for job in jobs_data:
    print(job)

# 5. 브라우저 종료
driver.quit()

{'title': 'Sr. Data Engineer', 'company_name': '잡플래닛', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1269623', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'python', 'aws', 'SQL', 'azure', '머신러닝', 'nosql', 'Spark', 'gcp', 'Tensorflow'], 'career': '5 ~ 13년', 'region': '서울', 'end_date': '2025.01.02'}
{'title': 'Machine Learning Engineer, R&D', 'company_name': '(주)클로버추얼패션', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1276446', 'platform_name': 'Jobplanet', 'stack': ['3d', '머신러닝', 'Docker', '딥러닝'], 'career': '2 ~ 10년', 'region': '서울', 'end_date': '2024.12.13'}
{'title': 'Data Engineer', 'company_name': '(주)자비스앤빌런즈', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284279', 'platform_name': 'Jobplanet', 'stack': ['Spark'], 'career': '4 ~ 20년', 'region': '서울', 'end_date': '2025.01.12'}
{'title': '[Riiid] Senior ML Engineer(5년 이상)', 'company_name': '(주)뤼이드', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284356', 'platform_name': 'Jobplanet', 'stack': ['AI', '머신러닝'], 'career': '5 ~ 10년', 'region': '서울', 'end_date': '2025.01.14'}
{'title': 'Senior Machine Learning Engineer', 'company_name': '(주)몰로코', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278093', 'platform_name': 'Jobplanet', 'stack': ['java', 'python', 'machine learning', 'Tensorflow', 'Golang'], 'career': '4 ~ 10년', 'region': '서울', 'end_date': '2024.12.27'}
{'title': '특허 빅데이터분석가 (IP빅데이터분석팀)', 'company_name': '(주)워트인텔리전스', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1270644', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', '빅데이터 분석'], 'career': '3년 이상', 'region': '서울', 'end_date': '2024.10.26'}
{'title': 'AI 엔지니어 (전문연구요원 신규/편입 가능)', 'company_name': '(주)워트인텔리전스', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1281619', 'platform_name': 'Jobplanet', 'stack': ['java', 'python', '머신러닝', '전문연구요원', '자연어처리', '알고리즘', '성능평가', '기술연구', 'LLM', 'MLOPS'], 'career': '3 ~ 20년', 'region': '서 울', 'end_date': '2024.10.29'}
{'title': 'Data Engineer(Jr)', 'company_name': '(주)데이터라이즈', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1279584', 'platform_name': 'Jobplanet', 'stack': ['python', '프로그래밍', 'SQL', '데이터엔지니어', '데이터수집', '전처리'], 'career': '1 ~ 2년', 'region': '서울', 'end_date': '2024.11.26'}
{'title': 'Data Analytics Engineer(Jr)', 'company_name': '(주)데이터라이즈', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1279583', 'platform_name': 'Jobplanet', 'stack': ['python', 'aws', 'SQL', '파이썬'], 'career': '1 ~ 2년', 'region': '서울', 'end_date': '2024.11.26'}
{'title': 'Data Engineer', 'company_name': '(주)데이터라이즈', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1277257', 'platform_name': 'Jobplanet', 'stack': ['python', '프로그래밍', 'SQL', '데이터엔지니어', '데이터수집', '전처리'], 'career': '3 ~ 10년', 'region': '서울', 'end_date': '2024.11.26'}
{'title': '[디지털헬스케어] AI 엔지니어 (병역특례 / CTO 레벨 가능)', 'company_name': '(주)엑소시스템즈', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280152', 'platform_name': 'Jobplanet', 'stack': ['C++', 'AI', '데이터분석', 'python', '딥러닝', '알고리즘 개발'], 'career': '경력무관', 'region': '경기', 'end_date': '2025.01.29'}       
{'title': '데이터사이언티스트 팀장', 'company_name': '(주)해줌', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1281678', 'platform_name': 'Jobplanet', 'stack': ['python', '머신러닝', '팀관리', '알고리즘 개발'], 'career': '5 ~ 20년', 'region': '서울', 'end_date': '2024.10.29'}
{'title': 'AI Language Model Engineer', 'company_name': '(주)알지에이아이엔씨', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278636', 'platform_name': 'Jobplanet', 'stack': ['C++', 'AI', 'python', 'machine learning', 'Deep Learning', 'Tensorflow', 'Pytorch'], 'career': '경력무관', 'region': '서울', 'end_date': '2025.01.01'}
{'title': 'Optimization Research Engineer', 'company_name': '(주)알지에이아이엔씨', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278634', 'platform_name': 'Jobplanet', 'stack': ['C++', 'AI', 'python', 'machine learning', 'Deep Learning', 'Tensorflow', 'Pytorch'], 'career': '경력무관', 'region': '서울', 'end_date': '2025.01.01'}       
{'title': '데이터 엔지니어', 'company_name': '(주)로민', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1283089', 'platform_name': 'Jobplanet', 'stack': ['python', 'PostgreSQL', '데이터관리', '성과분석', 'Docker', '시스템구축', 'airflow', 'boto3'], 'career': '3 ~ 15년', 'region': '서울', 'end_date': '2024.10.31'}
{'title': 'Data Scientist', 'company_name': '(주)에이비일팔공', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282238', 'platform_name': 'Jobplanet', 'stack': ['python', 'SQL', 'R', '머신러닝', 'machine learning', 'Tensorflow', 'Pytorch'], 'career': '경력무관', 'region': '서울', 'end_date': '2024.11.12'}
{'title': 'Research Scientist (Medical AI)', 'company_name': '(주)에이아이트릭스', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278565', 'platform_name': 'Jobplanet', 'stack': ['머신러닝', 'machine learning', '딥러닝', 'Deep Learning'], 'career': '5 ~ 15년', 'region': '서울', 'end_date': '2024.12.02'}
{'title': 'Machine Learning Engineer (전문연구요원), R&D', 'company_name': '(주)클로버추얼패션', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1276447', 'platform_name': 'Jobplanet', 'stack': ['3d', '머신러닝', 'Docker', '딥러닝'], 'career': '2 ~ 10년', 'region': '서울', 'end_date': '2024.12.13'}
{'title': 'AI 컴파일러 엔지니어 (전문연구요원 가능)', 'company_name': '(주)에너자이', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280545', 'platform_name': 'Jobplanet', 'stack': ['c', 'C++', 'AI', '알고리즘'], 'career': '경력무관', 'region': '서울', 'end_date': '2025.01.04'}
{'title': 'Robot Perception Engineer', 'company_name': '(주)알지에이아이엔씨', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278632', 'platform_name': 'Jobplanet', 'stack': ['C++', 'AI', 'python', 'machine learning', 'Deep Learning', 'Tensorflow', 'Pytorch'], 'career': '경력무관', 'region': '서울', 'end_date': '2025.01.01'}
{'title': '시니어 데이터 사이언티스트', 'company_name': '(주)아이넥스코퍼레이션', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280951', 'platform_name': 'Jobplanet', 'stack': ['C++', 'python', 'Linux', '딥러닝', 'opencv'], 'career': '5 ~ 12년', 'region': '서울', 'end_date': '2025.01.13'}
{'title': 'Staff Machine Learning Engineer', 'company_name': '(주)몰로코', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278094', 'platform_name': 'Jobplanet', 'stack': ['java', 'python', 'machine learning', 'Tensorflow', 'Golang'], 'career': '7 ~ 20년', 'region': '서울', 'end_date': '2024.12.27'}
{'title': '데이터 사이언티스트 (리드)', 'company_name': '(주)하이', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284153', 'platform_name': 'Jobplanet', 'stack': ['machine learning', 'restful api', 'Pytorch'], 'career': '7 ~ 10년', 'region': '서울', 'end_date': '2025.01.08'}
{'title': 'D2C Commerce Manager', 'company_name': '(주)넥스트챕터', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282421', 'platform_name': 'Jobplanet', 'stack': ['CRM', '데이터분석', 'CRM마케팅'], 'career': '3 ~ 10년', 'region': '서울', 'end_date': '2024.11.13'}
{'title': 'Runtime 소프트웨어 엔지니어 (전문연구요원 가능)', 'company_name': '(주)에너자이', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280546', 'platform_name': 'Jobplanet', 'stack': ['c', 'C++', 'AI', 'python', '머신러닝', '파이썬', '딥러닝'], 'career': '경력무관', 'region': '서울', 'end_date': '2025.01.04'}
{'title': '[광고 추천팀]ML Engineer/머신러닝 엔지니어 (신입, 병역특례요원 가능)', 'company_name': '(주)버즈빌', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284276', 'platform_name': 'Jobplanet', 'stack': ['Pytorch', 'AWS S3', 'sagemaker', 'athena', 'Mlflow', 'triton'], 'career': '경력무관', 'region': '서울', 'end_date': '2025.01.12'}
{'title': '데이터 엔지니어', 'company_name': '(주)워트인텔리전스', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1270640', 'platform_name': 'Jobplanet', 'stack': ['java', 'python', 'Linux', '파이썬'], 'career': '3년 이상', 'region': '서울', 'end_date': '2024.10.26'}
{'title': '빅데이터 솔루션 데이터 엔지니어(경력) 모집', 'company_name': '(주)로이드케이', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280816', 'platform_name': 'Jobplanet', 'stack': ['java', '빅데이터', 'python', 'aws', 'Linux', 'SQL', 'MYSQL', 'elasticsearch', '파이썬'], 'career': '5 ~ 13년', 'region': '서울', 'end_date': '2024.11.09'}
{'title': 'SAS 기반 데이터처리 전문가(경력 4년 이상)', 'company_name': '알테어엔지니어링(주)', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1277827', 'platform_name': 'Jobplanet', 'stack': ['데이터 분석', '금융권'], 'career': '4 ~ 15년', 'region': '경기', 'end_date': '2024.12.31'}
{'title': 'Data Analyst', 'company_name': '(주)클로버추얼패션', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1274059', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'SQL', 'Tableau', '문제해결능력', '비즈니스 영어'], 'career': '3 ~ 10년', 'region': '서울', 'end_date': '2024.10.17'}
{'title': '[Metheus] Research Engineer - CO', 'company_name': '(주)틸다', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282641', 'platform_name': 'Jobplanet', 'stack': ['AI', 'machine learning', 'Algorithm', 'Data Analytics'], 'career': '경력무관', 'region': '서울', 'end_date': '2024.10.31'}
{'title': 'Data Analyst(Team Lead)', 'company_name': '잡플래닛', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1269614', 'platform_name': 'Jobplanet', 'stack': ['data analysis', 'Data Analytics', 'Data visualization', 'Data modeling', 'Statistical Analysis', 'Data Mart Management', 'BusinessIntelligence'], 'career': '9 ~ 18년', 'region': '서울', 'end_date': '2025.01.02'}
{'title': 'DBA(5년 이상)', 'company_name': '(주)콜로세움코퍼레이션', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1281438', 'platform_name': 'Jobplanet', 'stack': ['빅데이터', 'SQL', 'dbms'], 'career': '5 ~ 10년', 'region': '서울', 'end_date': '2024.12.05'}
{'title': '브이캣 딥러닝 엔지니어', 'company_name': '(주)파이온코퍼레이션', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1273377', 'platform_name': 'Jobplanet', 'stack': ['java', 'python', '딥러닝', '알고리즘 개발'], 'career': '경력무관', 'region': '서울', 'end_date': '2024.11.08'}
{'title': '마케팅 Data Analyst (Junior)', 'company_name': '(주)바비톡', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282016', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', '마케팅분석'], 'career': '3 ~ 5년', 'region': '서울', 'end_date': '2025.01.06'}
{'title': 'Data Analytics Engineer', 'company_name': '(주)데이터라이즈', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1277397', 'platform_name': 'Jobplanet', 'stack': ['python', 'aws', 'SQL', '파이썬'], 'career': '2 ~ 10년', 'region': '서울', 'end_date': '2024.11.26'}
{'title': '데이터 엔지니어', 'company_name': '(주)하이퍼리즘', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280855', 'platform_name': 'Jobplanet', 'stack': ['java', '데이터분석', 'python', 'SQL', 'hadoop', 'nosql'], 'career': '경력무관', 'region': '서울', 'end_date': '2025.04.07'}
{'title': '[윌로그] 데이터 분석 담당자 리드(10년 이상)', 'company_name': '(주)윌로그', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1278333', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'python', 'SQL', '데이터모델링', '인사이트 도출'], 'career': '10 ~ 15년', 'region': '서울', 'end_date': '2025.01.13'}
{'title': '[UA팀] 모바일게임 데이터 분석가 (주니어)', 'company_name': '(주)에임드', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282533', 'platform_name': 'Jobplanet', 'stack': ['마케팅', '데이터분석', '게임', '데이터', '지표관리', '지표', '지표분석'], 'career': '1 ~ 3년', 'region': '서울', 'end_date': '2024.11.18'}
{'title': 'MLOps Engineer (LLM)', 'company_name': '(주)에이아이트릭스', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284040', 'platform_name': 'Jobplanet', 'stack': ['MLOPS'], 'career': '3 ~ 8년', 'region': '서울', 'end_date': '2025.01.05'}
{'title': '데이터 분석가', 'company_name': '(주)브릭', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280677', 'platform_name': 'Jobplanet', 'stack': ['python', 'R', 'machine learning', 'EDA', 'MLOPS'], 'career': '5 ~ 15년', 'region': '경기', 'end_date': '2025.01.06'}
{'title': 'ML Researcher (머신러닝 연구원)', 'company_name': '(주)메이아이', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1275872', 'platform_name': 'Jobplanet', 'stack': ['인공지능', 'AI', '머신러닝', 'ml'], 'career': '경력무관', 'region': '서울', 'end_date': '2024.10.31'}
{'title': 'Senior/Staff Data Scientist - Growth Analytics', 'company_name': '(주)몰로코', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1276603', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', '통계', 'python', 'SQL', '커뮤니케이션', '한국어', '비즈니스 영어'], 'career': '5 ~ 20년', 'region': '서울', 'end_date': '2024.11.13'}    
{'title': 'AI/빅데이터 분야 데이터 사이언티스트', 'company_name': '바이오에이아이', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282193', 'platform_name': 'Jobplanet', 'stack': ['빅데이터', '데이터분석', 'python', 'aws', 'SQL', '머신러닝', 'machine learning', '딥러닝', 'github', 'scikit-learn'], 'career': '3 ~ 20년', 'region': '서울', 'end_date': '2024.10.30'}
{'title': 'NLP Enigneer (LLM)', 'company_name': '(주)에이아이트릭스', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284039', 'platform_name': 'Jobplanet', 'stack': ['NLP', 'LLM'], 'career': '2 ~ 10년', 'region': '서울', 'end_date': '2025.01.05'}
{'title': 'Data Engineer', 'company_name': '에프피티소프트웨어코리아(주)', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282096', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'python', 'Linux', 'SQL', 'R', 'MYSQL', '파이썬', '딥러닝', 'Tableau', 'airflow'], 'career': '3 ~ 10년', 'region': '서울', 'end_date': '2024.11.05'}     
{'title': 'Data Analyst(경력 7년 이상)', 'company_name': '(주)오픈서베이', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1283435', 'platform_name': 'Jobplanet', 'stack': ['python', 'SQL', 'R', 'Tableau'], 'career': '7 ~ 15년', 'region': '서울', 'end_date': '2024.10.18'}
{'title': '제품 Data Analyst (Senior)', 'company_name': '(주)바비톡', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282385', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'SQL'], 'career': '7 ~ 9년', 'region': '서울', 'end_date': '2025.01.06'}
{'title': '제품 Data Analyst (Middle)', 'company_name': '(주)바비톡', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282390', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'SQL'], 'career': '5 ~ 7년', 'region': '서울', 'end_date': '2025.01.06'}
{'title': '데이터 분석가', 'company_name': '(주)뮤즈라이브', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1281663', 'platform_name': 'Jobplanet', 'stack': ['python', 'SQL', 'google analytics', '인사이트 도출', 'firebase', 'amplitude'], 'career': '2 ~ 10년', 'region': '서울', 'end_date': '2024.11.21'}
{'title': '디지털헬스케어 AI 엔지니어 (병역특례 가능)', 'company_name': '(주)엑소시스템즈', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1273524', 'platform_name': 'Jobplanet', 'stack': ['C++', 'AI', 'python'], 'career': '경력무관', 'region': '경기', 'end_date': '2024.11.08'}
{'title': 'Data Analyst(Jr)', 'company_name': '(주)데이터라이즈', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282808', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'dw', 'Data analyst'], 'career': '신입', 'region': '서울', 'end_date': '2024.11.26'}
{'title': 'Head of Credit Risk and Data', 'company_name': '(주)에잇퍼센트', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280912', 'platform_name': 'Jobplanet', 'stack': ['python', 'Cloud'], 'career': '10 ~ 20년', 'region': '서울', 'end_date': '2025.01.08'}
{'title': 'Onchain Data Analyst (퀀트 리서처)', 'company_name': '(주)하이퍼리즘', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280858', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'python', 'R', 'Tableau', 'POWER BI', 'etherscan'], 'career': '경력무관', 'region': '서울', 'end_date': '2025.04.07'}
{'title': 'Data Scientist (경력 3년~)', 'company_name': '(주)한국그린데이터', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1273395', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'python', 'SQL', 'R', '머신러닝', '데이터 분석', '파이썬', '딥러닝', 'Tensorflow', 'Pytorch'], 'career': '경력무관', 'region': '경기', 'end_date': '2024.11.08'}
{'title': '[윌로그] 데이터 분석 담당자 (3년 이상)', 'company_name': '(주)윌로그', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1279482', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'python', 'SQL', '전략수립', '데이터모델링', '팀관리', '인사이트 도출', 'ml'], 'career': '3 ~ 6년', 'region': '서울', 'end_date': '2025.01.13'}   
{'title': 'AI Engineer', 'company_name': '(주)모플', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1279413', 'platform_name': 'Jobplanet', 'stack': ['python', '데이터 분석 능력', '수학적 사고', '컴퓨터 및 소프트웨어 이해도'], 'career': '경력무관', 'region': '서울', 'end_date': '2024.12.11'}
{'title': 'Senior Data Scientist - Ads Performance', 'company_name': '(주)몰로코', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1273316', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'python', 'SQL', 'R', '머신러닝', '데이터 분석', '파이썬', '딥러닝', '데이터과학자'], 'career': '3 ~ 20년', 'region': '서울', 'end_date': '2024.11.08'}
{'title': 'Marketing Data Scientist', 'company_name': '(주)에이비일팔공', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282345', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'BI', 'python', 'SQL', 'R', '파이썬'], 'career': '경력무관', 'region': '서울', 'end_date': '2024.11.19'}
{'title': 'Data Engineer', 'company_name': '(주)모플', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1279411', 'platform_name': 'Jobplanet', 'stack': ['Oracle', 'PostgreSQL', 'nosql', 'Spark', 'MongoDB', 'RDB', 'airflow'], 'career': '2 ~ 8년', 'region': '서울', 'end_date': '2024.12.11'}
{'title': 'Data Analyst', 'company_name': '(주)데이터라이즈', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1277398', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'dw', 'Data analyst'], 'career': '2 ~ 10년', 'region': '서울', 'end_date': '2024.11.26'}
{'title': 'Data Analyst', 'company_name': '(주)자비스앤빌런즈', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282626', 'platform_name': 'Jobplanet', 'stack': ['머신러닝'], 'career': '2 ~ 10년', 'region': '서울', 'end_date': '2025.01.12'}
{'title': '마케팅 Data Analyst (Middle)', 'company_name': '(주)바비톡', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1281227', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', '마케팅분석'], 'career': '5 ~ 7년', 'region': '서울', 'end_date': '2025.01.06'}
{'title': 'Data Analyst', 'company_name': '잡플래닛', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1283924', 'platform_name': 'Jobplanet', 'stack': ['데이 터분석', 'python', 'SQL', 'R'], 'career': '5 ~ 15년', 'region': '서울', 'end_date': '2025.01.09'}
{'title': '데이터 분석가(3년 이상)', 'company_name': '(주)버즈빌', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1283913', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'python', 'SQL', 'Tableau'], 'career': '3 ~ 10년', 'region': '서울', 'end_date': '2024.12.31'}
{'title': 'ML(Data) Engineer', 'company_name': '(주)클로잇', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282585', 'platform_name': 'Jobplanet', 'stack': ['AI', 'Kubernetes', 'ml', 'MLOPS', 'CSP'], 'career': '3 ~ 20년', 'region': '서울', 'end_date': '2024.11.19'}
{'title': 'GA/AA데이터 분석(사원대리과장급)', 'company_name': '콘센트릭스서비스코리아(유)', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1267923', 'platform_name': 'Jobplanet', 'stack': ['QA', '영어', 'google analytics', 'adobe analytics', '태깅'], 'career': '1 ~ 11년', 'region': '서울', 'end_date': '2025.01.13'}
{'title': 'Data Analyst', 'company_name': '(주)아임웹', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1274331', 'platform_name': 'Jobplanet', 'stack': ['데 이터분석', 'python', 'SQL', '데이터관리', '데이터베이스'], 'career': '4 ~ 10년', 'region': '서울', 'end_date': '2024.11.03'}
{'title': '[미리디] 데이터 엔지니어', 'company_name': '(주)미리디', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1283068', 'platform_name': 'Jobplanet', 'stack': ['java', 'python', 'SQL'], 'career': '3 ~ 7년', 'region': '서울', 'end_date': '2024.12.02'}
{'title': '데이터분석(태깅)', 'company_name': '콘센트릭스서비스코리아(유)', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1283871', 'platform_name': 'Jobplanet', 'stack': ['ga', 'AA'], 'career': '1 ~ 13년', 'region': '서울', 'end_date': '2024.12.29'}
{'title': '[쿠팡] 데이터 분석 & 테스트 PM', 'company_name': '쿠팡(주)', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284003', 'platform_name': 'Jobplanet', 'stack': ['영어', '데이터분석', 'python', 'SQL', '파이썬'], 'career': '8년 이상', 'region': '서울', 'end_date': '2024.10.31'}
{'title': 'Sr. Analyst/Manager of Program Risk Management', 'company_name': '아슈리온코리아(유)', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1269546', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'python', 'SQL', '머신러닝', '딥러닝', 'communication', 'Presentation', 'Ms Excel'], 'career': '7 ~ 12년', 'region': '서울', 'end_date': '2025.01.27'}
{'title': '[쿠팡] 로지스틱스 프로세스 개선 담당자 (Logistics Excellence Innovation)', 'company_name': '쿠팡(주)', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284023', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'SQL', 'Pytorch'], 'career': '신입', 'region': '서울', 'end_date': '2024.10.31'}
{'title': '[외국인 채용] VoC 데이터분석(신입/경력)', 'company_name': '기아(주)', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1283956', 'platform_name': 'Jobplanet', 'stack': ['빅데이터', '데이터분석'], 'career': '경력무관', 'region': '서울', 'end_date': '2024.10.21'}
{'title': '[쿠팡] 물류 기획 데이터 담당자 (Capacity Strategy)', 'company_name': '쿠팡(주)', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284022', 'platform_name': 'Jobplanet', 'stack': ['엑셀', '데이터분석', 'python', 'SQL', '구글 스프레드 시트'], 'career': '1년 이상', 'region': '서울', 'end_date': '2024.10.31'}
{'title': '데이터 엔지니어', 'company_name': '슈퍼센트(주)', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284110', 'platform_name': 'Jobplanet', 'stack': ['python', 'SQL', 'Docker', 'typescript', 'chatGPT', 'GDC 인프라'], 'career': '경력무관', 'region': '서울', 'end_date': '2025.01.12'}
{'title': '[미리디] 데이터 분석', 'company_name': '(주)미리디', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284139', 'platform_name': 'Jobplanet', 'stack': ['데이터분석', 'python', 'SQL', 'R', '파이썬', 'Tableau'], 'career': '1 ~ 5년', 'region': '서울', 'end_date': '2025.01.08'}
{'title': '[쿠팡풀필먼트서비스] 데이터 분석가 (Data Analyst)', 'company_name': '쿠팡(주)', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1284021', 'platform_name': 'Jobplanet', 'stack': ['빅데이터', '데이터분석', 'python', 'R', 'Spark', '딥러닝', 'github', 'airflow'], 'career': '5년 이상', 'region': '서울', 'end_date': '2024.10.31'}    
{'title': '[경력/5년 이상] 데이터 엔지니어', 'company_name': '(주)오아시스비즈니스', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1280334', 'platform_name': 'Jobplanet', 'stack': ['python', 'aws', 'MYSQL', 'PostgreSQL', 'kafka', 'gcp', 'Tableau', 'airflow', 'GCP BigQuery'], 'career': '5 ~ 7년', 'region': '서울', 'end_date': '2024.10.31'}
{'title': 'Data Analyst', 'company_name': '번개장터(주)', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1281904', 'platform_name': 'Jobplanet', 'stack': ['python', 'SQL', 'R'], 'career': '3 ~ 6년', 'region': '서울', 'end_date': '2024.10.31'}
{'title': '[피알원]데이터 애널리스트(리서쳐) 경력직', 'company_name': '(주)피알원', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1282804', 'platform_name': 'Jobplanet', 'stack': ['전략기획', '시장조사', '설문조사', '빅데이터분석'], 'career': '1 ~ 6년', 'region': '서울', 'end_date': '2024.11.26'}
{'title': '[Global Talent(Foreigner)] VoC Data Analysis(Entry/Experienced)', 'company_name': '기아(주)', 'detail_url': 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1283957', 'platform_name': 'Jobplanet', 'stack': ['빅데이터', '데이터분석'], 'career': '경력무관', 'region': '서울', 'end_date': '2024.10.21'}

{'title': '데이터 사이언티스', 'company_name': '스펙터', 'detail_url': 'https://www.wanted.co.kr/wd/245600', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-15년', 'region': '테헤란로7길 22', 'stack': ['Python', 'R', 'Power BI', 'Tableau']}
{'title': 'Data Engineer (3-5년)', 'company_name': '크몽(kmong)', 'detail_url': 'https://www.wanted.co.kr/wd/245341', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-5년', 'region': '서울시 서초구 사임당로 157, 릿타워 3층 (주)크몽', 'stack': ['Python', 'AWS']}
{'title': 'Data Scientist', 'company_name': '하이퍼커넥트', 'detail_url': 'https://www.wanted.co.kr/wd/245455', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-15년', 'region': '강남구 영동대로 517 아셈타워', 'stack': ['기술 스택 없음']}
{'title': '빅데이터 솔루션 (Elastic) 영업담당자', 'company_name': '위앤유텍', 'detail_url': 'https://www.wanted.co.kr/wd/244993', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-7년', 'region': '서울특별시 송파구 백제고분로 75 311,312호', 'stack': ['기술 스택 없음']}
{'title': '데이터 매니저', 'company_name': '스누아이랩', 'detail_url': 'https://www.wanted.co.kr/wd/245433', 'platform_name': 'wanted', 'end_date': '2025.01.15', 'career': '경력 1-5년', 'region': '수원시 영통구 광교로 145 차세대융합기술연구원 A동 1202호', 'stack': ['기술 스택 없음']}
{'title': '[Data Exchange]Project Manager', 'company_name': '앤서스랩코리아', 'detail_url': 'https://www.wanted.co.kr/wd/245130', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울특별시 강남구 언주 로 609, PAX TOWER 4층', 'stack': ['기술 스택 없음']}
{'title': 'Data Engineer (5년 이상)', 'company_name': '알스퀘어', 'detail_url': 'https://www.wanted.co.kr/wd/244076', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-20년', 'region': '서울 강남구 테헤란로 311(아남타워)', 'stack': ['기술 스택 없음']}
{'title': 'Cloud Sales Leader (MSP사 총괄 임원급)', 'company_name': '어반데이터랩', 'detail_url': 'https://www.wanted.co.kr/wd/245243', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 12-16년', 'region': '서울시 종로구 창경궁로 240-7 2층', 'stack': ['기술 스택 없음']}
{'title': 'Data Engineer (3-5년)', 'company_name': '크몽(kmong)', 'detail_url': 'https://www.wanted.co.kr/wd/245341', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-5년', 'region': '서울시 서초구 사임당로 157, 릿타워 3층 (주)크몽', 'stack': ['Python', 'AWS']}
{'title': '[Data Exchange]Project Manager', 'company_name': '앤서스랩코리아', 'detail_url': 'https://www.wanted.co.kr/wd/245130', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울특별시 강남구 언주 로 609, PAX TOWER 4층', 'stack': ['기술 스택 없음']}
{'title': '빅데이터 솔루션 (Elastic) 영업담당자', 'company_name': '위앤유텍', 'detail_url': 'https://www.wanted.co.kr/wd/244993', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-7년', 'region': '서울특별시 송파구 백제고분로 75 311,312호', 'stack': ['기술 스택 없음']}
{'title': '데이터 매니저', 'company_name': '스누아이랩', 'detail_url': 'https://www.wanted.co.kr/wd/245433', 'platform_name': 'wanted', 'end_date': '2025.01.15', 'career': '경력 1-5년', 'region': '수원시 영통구 광교로 145 차세대융합기술연구원 A동 1202호', 'stack': ['기술 스택 없음']}
{'title': 'Data Scientist', 'company_name': '하이퍼커넥트', 'detail_url': 'https://www.wanted.co.kr/wd/245455', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-15년', 'region': '강남구 영동대로 517 아셈타워', 'stack': ['기술 스택 없음']}
{'title': '데이터 사이언티스', 'company_name': '스펙터', 'detail_url': 'https://www.wanted.co.kr/wd/245600', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-15년', 'region': '테헤란로7길 22', 'stack': ['Python', 'R', 'Power BI', 'Tableau']}
{'title': 'Data Engineer (5년 이상)', 'company_name': '알스퀘어', 'detail_url': 'https://www.wanted.co.kr/wd/244076', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-20년', 'region': '서울 강남구 테헤란로 311(아남타워)', 'stack': ['기술 스택 없음']}
{'title': 'Cloud Sales Leader (MSP사 총괄 임원급)', 'company_name': '어반데이터랩', 'detail_url': 'https://www.wanted.co.kr/wd/245243', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 12-16년', 'region': '서울시 종로구 창경궁로 240-7 2층', 'stack': ['기술 스택 없음']}
{'title': 'Data Analyst', 'company_name': '프렉스코리아(Prex)', 'detail_url': 'https://www.wanted.co.kr/wd/245158', 'platform_name': 'wanted', 'end_date': '2024.11.01', 'career': '경력 4-9년', 'region': '서울특별시 영등포구 국제금융로 10, Two IFC, 7층', 'stack': ['기술 스택 없음']}
{'title': 'DW/BI/ETL 데이터 개발자(4년이상)', 'company_name': '부드러운돌멩이', 'detail_url': 'https://www.wanted.co.kr/wd/244420', 'platform_name': 'wanted', 'end_date': '2024.11.30', 'career': '경력 4-10년', 'region': '서울 성동구 연무장5길 9-16 (성수동2가, 블루스톤타워) 10층 1003호 부드러운돌멩이', 'stack': ['기술 스택 없음']}
{'title': '[KMX] VAN 사업운영 지원 담당자 (junior)', 'company_name': '한국신용데이터(KCD)', 'detail_url': 'https://www.wanted.co.kr/wd/245088', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-5년', 'region': '서울특별시 강남구 테헤란로 7길 7, 에스코빌딩 10층', 'stack': ['기술 스택 없음']}
{'title': '데이터 사이언티스트 (5년 이상)', 'company_name': '피엠그로우', 'detail_url': 'https://www.wanted.co.kr/wd/242343', 'platform_name': 'wanted', 'end_date': '2024.10.20', 'career': '경력 5년 이상', 'region': '서울시 서초구 반포대로 23길 6 효진빌딩 5층', 'stack': ['Python', 'SQL', '딥 러닝', '개발', '모델링', 'ML']}
{'title': '데이터 사이언티스트(부피 알고리즘)', 'company_name': '안티그래비티', 'detail_url': 'https://www.wanted.co.kr/wd/105496', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2년 이상', 'region': '서울 성동구 성수이 로22길 37 (성수동2가) 아크밸리 지식산업 센터 303c호 안티그래비티', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '이노케어플러스', 'detail_url': 'https://www.wanted.co.kr/wd/244681', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3년 이상', 'region': '서울특별시 서초구 효령로 107, 4층(방배동)', 'stack': ['SQL', 'Tableau', 'AWS', 'Spotfire']}
{'title': '플랫폼 개발자 풀스택 (12년 이상/팀장급)', 'company_name': '어반데이터랩', 'detail_url': 'https://www.wanted.co.kr/wd/233965', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 12-16년', 'region': '서울시 종로구  창경궁로 240-7 2층', 'stack': ['기술 스택 없음']}
{'title': 'Data Engineer', 'company_name': '뤼이드(Riiid)', 'detail_url': 'https://www.wanted.co.kr/wd/216750', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-3년', 'region': '강남구 테헤란로 424 삼성생명 대치타워 19F', 'stack': ['Github', 'Azure', 'Spark', 'Python', 'SQL', 'AWS', 'GCP', 'Kubernetes']}
{'title': '데이터사업본부 Pre-Sales', 'company_name': '크라우드웍스', 'detail_url': 'https://www.wanted.co.kr/wd/241178', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 10-15년', 'region': '서울특별시 강남구 역삼동 테헤 란로 309 (삼성제일빌딩, 5층)', 'stack': ['기술 스택 없음']}
{'title': '데이터 사이언티스트', 'company_name': '안티그래비티', 'detail_url': 'https://www.wanted.co.kr/wd/147683', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 1-20년', 'region': '서울 성동구 성수이로22길 37 (성수동2가) 아크밸리 지식산업 센터 303c호 안티그래비티', 'stack': ['기술 스택 없음']}
{'title': '데이터 사이언티스트', 'company_name': '베이글코드(Bagelcode)', 'detail_url': 'https://www.wanted.co.kr/wd/241366', 'platform_name': 'wanted', 'end_date': '2024.10.31', 'career': '경력 3-5년', 'region': '서울특별시 강남구 테헤란로328 (역삼동 708-1) 동우빌딩 16층', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '알파브릿지', 'detail_url': 'https://www.wanted.co.kr/wd/244087', 'platform_name': 'wanted', 'end_date': '2024.10.31', 'career': '경력 3-10년', 'region': '경기도 성남시 분당구 판교역로 192번길 14,9층(삼평동, 리치투게더센터)', 'stack': ['기술 스택 없음']}
{'title': 'Data Engineer', 'company_name': '아임웹', 'detail_url': 'https://www.wanted.co.kr/wd/240941', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울시 강남구 테헤란로 501 VPLEX 14층 아임웹 강 남 오피스', 'stack': ['기술 스택 없음']}
{'title': 'CBO (Chief Business Officer)', 'company_name': '데이터뱅크', 'detail_url': 'https://www.wanted.co.kr/wd/141458', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '강남구 테헤란로86길 12, 2층', 'stack': ['기술 스택 없음']}
{'title': 'B2B 영업 관리 (통번역 및 데이터)', 'company_name': '업템포글로벌', 'detail_url': 'https://www.wanted.co.kr/wd/232680', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3년 이상', 'region': '서울시 강남구 선릉로561, 2층', 'stack': ['기술 스택 없음']}
{'title': '빅데이터 기반 AI 분석 시스템 구축 전문가', 'company_name': '버블콘', 'detail_url': 'https://www.wanted.co.kr/wd/243869', 'platform_name': 'wanted', 'end_date': '2024.10.31', 'career': '경력 2-10년', 'region': '구로구 디지털로34길 43, 1001호', 'stack': ['기술 스택 없음']}
{'title': '[Data Exchange]시스템 기획', 'company_name': '앤서스랩코리아', 'detail_url': 'https://www.wanted.co.kr/wd/245131', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울특별시 강남구 언주로 609, PAX TOWER 4층', 'stack': ['기술 스택 없음']}
{'title': '[BM]Marketing Data Engineer', 'company_name': '앤서스랩코리아', 'detail_url': 'https://www.wanted.co.kr/wd/245120', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7-15년', 'region': '서울특별시 강남구 언주로 609, PAX TOWER 4층', 'stack': ['기술 스택 없음']}
{'title': '기업부설연구소 데이터 사이언티스트', 'company_name': '에스앤이컴퍼니', 'detail_url': 'https://www.wanted.co.kr/wd/244181', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울특별시 서초구  매헌로8길 39, D동 406호', 'stack': ['Python']}
{'title': 'AI/빅데이터 분야 데이터 사이언티스트', 'company_name': '바이오에이아이', 'detail_url': 'https://www.wanted.co.kr/wd/195819', 'platform_name': 'wanted', 'end_date': '2024.10.30', 'career': '경력 3-20년', 'region': '서울 강남구 남 부순환로 2621', 'stack': ['Git', 'Github', 'MySQL', 'Pytorch', 'Scikit-Learn', 'Python', 'SQL', 'AWS', 'Docker', 'Kubeflow']}
{'title': '컴퓨터비전 분야 연구/개발자.', 'company_name': '어반데이터랩', 'detail_url': 'https://www.wanted.co.kr/wd/238723', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3년 이상', 'region': '서울시 종로구 창경궁로 240-7 2층', 'stack': ['Python', 'C', 'C++']}
{'title': 'Data Engineer', 'company_name': '와드(캐치테이블)', 'detail_url': 'https://www.wanted.co.kr/wd/199962', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '경기도 성남시 분당구 대왕판교로 660 유스페이스1 A동 605호', 'stack': ['MySQL', 'Python', 'SQL', 'Data Analysis', 'Tableau']}
{'title': 'Data Analyst(7년 이상)', 'company_name': '오픈서베이', 'detail_url': 'https://www.wanted.co.kr/wd/241880', 'platform_name': 'wanted', 'end_date': '2024.11.22', 'career': '경력 7년 이상', 'region': '서울특별시 강남구 강남대로84길 13 KR타워', 'stack': ['기술 스택 없음']}
{'title': 'SAS 기반 데이터처리 전문가(4년 이상)', 'company_name': '알테어엔지니어링', 'detail_url': 'https://www.wanted.co.kr/wd/223227', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 4년 이상', 'region': '경기도 성남시 분당구 황새울로 216, 6층 알테어', 'stack': ['React', 'SAS']}
{'title': '데이터 분석가(DA)', 'company_name': '에코마케팅', 'detail_url': 'https://www.wanted.co.kr/wd/244548', 'platform_name': 'wanted', 'end_date': '2024.10.20', 'career': '경력 3-6년', 'region': '서울 송파구 올림픽로35다길 42', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '로민', 'detail_url': 'https://www.wanted.co.kr/wd/243445', 'platform_name': 'wanted', 'end_date': '2024.10.31', 'career': '경력 3-15년', 'region': '서울 서초구 방배천로2길 10, 7층', 'stack': ['Git', 'Python', 'AWS', 'Docker', 'PostgreSQL']}
{'title': 'Data Engineer(3년 이상)', 'company_name': '비에이치에스엔', 'detail_url': 'https://www.wanted.co.kr/wd/240969', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년 이상', 'region': '서울 강남구 삼성로 570, 5층', 'stack': ['Github', 'Linux', 'MySQL', 'Python', 'AWS', 'Docker', 'GCP', 'RabbitMQ', 'Kubernetes', 'FastAPI']}
{'title': 'Cloud Sales Manager (MSP사)', 'company_name': '어반데이터랩', 'detail_url': 'https://www.wanted.co.kr/wd/237972', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입', 'region': '서울시 종로구 창경궁로 240-7 2층', 'stack': ['기술 스택 없음']}
{'title': 'Product Data Analyst', 'company_name': '아우름플래닛(Liner)', 'detail_url': 'https://www.wanted.co.kr/wd/85810', 'platform_name': 'wanted', 'end_date': '2024.12.31', 'career': '경력 3-10년', 'region': '서울특별시 마포구 양화로 140, 8층', 'stack': ['Pytorch', 'Tensorflow', 'Kubeflow']}
{'title': 'Web3 데이터 엔지니어', 'company_name': '팀블랙버드(CryptoQuant)', 'detail_url': 'https://www.wanted.co.kr/wd/128026', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-10년', 'region': '서울특별시 영등포구 국회대로 612', 'stack': ['Git', 'Flask', 'Google Cloud Platform', 'Hadoop', 'Redis', 'Spark', 'Python', 'AWS', 'Docker', 'PostgreSQL', 'GCP', 'Kubernetes']}
{'title': '데이터 엔지니어', 'company_name': '바로팜', 'detail_url': 'https://www.wanted.co.kr/wd/243708', 'platform_name': 'wanted', 'end_date': '2024.10.31', 'career': '경력 5-12년', 'region': '역삼동 명우빌딩 3층', 'stack': ['기술 스택  없음']}
{'title': '데이터 엔지니어', 'company_name': '미리디', 'detail_url': 'https://www.wanted.co.kr/wd/241731', 'platform_name': 'wanted', 'end_date': '2024.10.18', 'career': '경력 3-7년', 'region': '디지털로31길 12, 8층, 13층, 14층 (구로동, 태 평양물산)', 'stack': ['기술 스택 없음']}
{'title': 'Data Scientist (전문연구요원 가능)', 'company_name': '에니아이(Aniai)', 'detail_url': 'https://www.wanted.co.kr/wd/244307', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울 성동구 상원1 길 22 굿타워', 'stack': ['Python']}
{'title': '[기술본부] Data Analytics Engineer', 'company_name': '데브시스터즈(Devsisters)', 'detail_url': 'https://www.wanted.co.kr/wd/238131', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-20년', 'region': '서울특별 시 강남구 신사동 도산대로 327', 'stack': ['기술 스택 없음']}
{'title': '[잡플래닛] Data Analyst', 'company_name': '브레인커머스', 'detail_url': 'https://www.wanted.co.kr/wd/244040', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '서울 강남구 테헤란로 509 10층', 'stack': ['Python', 'R', 'SQL']}
{'title': 'Senior Data Scientist (3년 이상)', 'company_name': '데이블(Dable)', 'detail_url': 'https://www.wanted.co.kr/wd/237497', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3년 이상', 'region': '서울시 강남구 테헤란로 114길 38, 동일타워 1층', 'stack': ['기술 스택 없음']}
{'title': '[데이터사업부]제조실행시스템 개발_MES(JAVA.JSP)', 'company_name': '위즈코어', 'detail_url': 'https://www.wanted.co.kr/wd/208593', 'platform_name': 'wanted', 'end_date': '2024.10.26', 'career': '경력 3-15년', 'region': '서울시 성 동구 아차산로17길 49 성수생각공장 데시앙플렉스 1505-8호', 'stack': ['기술 스택 없음']}
{'title': '데이터엔지니어 (5년 이상)', 'company_name': '카테노이드', 'detail_url': 'https://www.wanted.co.kr/wd/242819', 'platform_name': 'wanted', 'end_date': '2024.10.31', 'career': '경력 5-10년', 'region': '서울시 강남구 봉은사로 502 삼 하빌딩 4층 5층', 'stack': ['기술 스택 없음']}
{'title': '[그루비 SaaS 솔루션] 데이터 사이언티스트', 'company_name': '플래티어', 'detail_url': 'https://www.wanted.co.kr/wd/157407', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 1-6년', 'region': '서울시 송파구 법원로9길 26  H Business Park D동 6층', 'stack': ['기술 스택 없음']}
{'title': 'Data Engineer (Backend)', 'company_name': '베이글코드(Bagelcode)', 'detail_url': 'https://www.wanted.co.kr/wd/236184', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-7년', 'region': '서울특별시 강남구 테헤란로328 (역삼동 708-1) 동우빌딩 16층', 'stack': ['기술 스택 없음']}
{'title': '신소재 및 신물질 AI/빅데이터 분야 데이터 팀장', 'company_name': '바이오에이아이', 'detail_url': 'https://www.wanted.co.kr/wd/195820', 'platform_name': 'wanted', 'end_date': '2024.10.30', 'career': '경력 5-20년', 'region': '서울  강남구 남부순환로 2621', 'stack': ['Github', 'MySQL', 'Python', 'SQL', 'AWS', 'Docker']}
{'title': 'Performance Marketing Data Manager', 'company_name': '하이퍼커넥트', 'detail_url': 'https://www.wanted.co.kr/wd/245019', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 6년 이상', 'region': '강남구 영동대로 517 아셈타워', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '베이글코드(Bagelcode)', 'detail_url': 'https://www.wanted.co.kr/wd/236189', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-7년', 'region': '서울특별시 강남구 테헤란로328 ( 역삼동 708-1) 동우빌딩 16층', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '딥노이드(Deepnoid)', 'detail_url': 'https://www.wanted.co.kr/wd/231814', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3년 이상', 'region': '디지털로33길 55, 904호 (구로동, 이앤씨벤처드림타워2차)', 'stack': ['Git', 'MongoDB', 'Spark', 'Python', 'Docker', 'PostgreSQL']}
{'title': 'Data Scientist', 'company_name': '우주로테크', 'detail_url': 'https://www.wanted.co.kr/wd/242697', 'platform_name': 'wanted', 'end_date': '2024.10.24', 'career': '경력 3년', 'region': '서울특별시 종로구 종로 6 5층', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '티비유', 'detail_url': 'https://www.wanted.co.kr/wd/233422', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2년 이상', 'region': '서울특별시 서초구 서초대로 78길 26, 10층 (강남선인빌딩)', 'stack': ['Java', 'Kotlin', 'Python', 'NoSQL']}
{'title': '금융 데이터 엔지니어', 'company_name': '쿠파', 'detail_url': 'https://www.wanted.co.kr/wd/240266', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-7년', 'region': '서울시 서초구 강남대로 373', 'stack': ['기술 스택 없음']}
{'title': '마이데이터 운영SM (7년 이상)', 'company_name': '티사이언티픽', 'detail_url': 'https://www.wanted.co.kr/wd/243662', 'platform_name': 'wanted', 'end_date': '2024.10.30', 'career': '경력 7년 이상', 'region': '서울특별시 마포구 월드 컵북로 416, LG U+ 상암사옥', 'stack': ['기술 스택 없음']}
{'title': 'Sales Manager', 'company_name': '데이터라이즈', 'detail_url': 'https://www.wanted.co.kr/wd/224575', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-6년', 'region': '서울특별시 강남구 도곡로 205, YK빌딩 6층', 'stack': ['기술 스택 없음']}
{'title': '데이터 사이언티스트 (리드)', 'company_name': '하이(Haii)', 'detail_url': 'https://www.wanted.co.kr/wd/244990', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7-15년', 'region': '종로구 삼일대로 428, 낙원상가 501호', 'stack': ['기술 스택 없음']}
{'title': '데이터사이언스 팀장', 'company_name': '해줌', 'detail_url': 'https://www.wanted.co.kr/wd/243806', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-20년', 'region': '서울시 송파구 법원로 128, SK V1 A동 706호', 'stack': ['Pytorch', 'Tensorflow', 'Python']}
{'title': 'Data Quality PM (Autonomous Driving)', 'company_name': '포티투닷(42dot)', 'detail_url': 'https://www.wanted.co.kr/wd/180662', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 4-13년', 'region': '경기 성남시 수정구 창업로40번길 20, A동 42dot', 'stack': ['기술 스택 없음']}
{'title': '[잡플래닛] Data Analyst(Team Lead)', 'company_name': '브레인커머스', 'detail_url': 'https://www.wanted.co.kr/wd/244625', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 9-15년', 'region': '서울 강남구 테헤란로 509 10층', 'stack': ['Data Analysis']}
{'title': 'Database Analyst(디지털분야특화, 5년 이상)', 'company_name': '인스파이어인티그레이티드리조트', 'detail_url': 'https://www.wanted.co.kr/wd/243361', 'platform_name': 'wanted', 'end_date': '2024.10.18', 'career': '경력 5년 이상', 'region': '인천광역시 중구 공항문화로 127(운서동, 인스파이어)', 'stack': ['기술 스택 없음']}
{'title': '데이터 분석가 리더(7년이상/서울)', 'company_name': '트리노드', 'detail_url': 'https://www.wanted.co.kr/wd/237100', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7-15년', 'region': '서울시 강남구 선릉로 667,  라이즈스퀘어 8층', 'stack': ['기술 스택 없음']}
{'title': '데이터베이스 관리자(10년 이상)', 'company_name': '아파트아이', 'detail_url': 'https://www.wanted.co.kr/wd/234574', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 10-15년', 'region': '서울특별시 금천구 가산디지털1로 186 제이플라츠 1203호', 'stack': ['기술 스택 없음']}
{'title': 'Data Analyst(2년 이상)', 'company_name': '데이터라이즈', 'detail_url': 'https://www.wanted.co.kr/wd/177600', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-10년', 'region': '서울특별시 강남구 도곡로 205, YK 빌딩 6층', 'stack': ['Data Analysis', 'Excel', 'Notion']}
{'title': '데이터분석팀 (시니어)', 'company_name': '로앤컴퍼니(lawtalk)', 'detail_url': 'https://www.wanted.co.kr/wd/231076', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 6-10년', 'region': '서울 강남구 테헤란로 420 KT선릉타워 West 19층 (선릉역 부근 - 1층이 대신증권)', 'stack': ['Python', 'SQL', 'Amplitude']}
{'title': '데이터분석가', 'company_name': '부동산플래닛', 'detail_url': 'https://www.wanted.co.kr/wd/245395', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입 이상', 'region': '서울특별시 강남구 영동대로 702 (화천빌딩) 7층', 'stack': ['기술 스택 없음']}
{'title': 'Data Scientist (5년이상)', 'company_name': '피플리', 'detail_url': 'https://www.wanted.co.kr/wd/237897', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년 이상', 'region': '마포구 월드컵북로6길 18', 'stack': ['기술 스택 없음']}
{'title': '[잡플래닛] Sr. Data Engineer', 'company_name': '브레인커머스', 'detail_url': 'https://www.wanted.co.kr/wd/117898', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-13년', 'region': '서울 강남구 테헤란로 509 10층', 'stack': ['Azure', 'AWS', 'GCP']}
{'title': '[아이디어스] 데이터 엔지니어', 'company_name': '백패커', 'detail_url': 'https://www.wanted.co.kr/wd/216500', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-7년', 'region': '서울특별시 서초구 서초대로 398 BNK디지털타워 19층, 20층', 'stack': ['기술 스택 없음']}
{'title': '[인텔리전스랩스] 데이터 플랫폼 PM (데이터 아키텍트)', 'company_name': '넥슨코리아(NEXON)', 'detail_url': 'https://www.wanted.co.kr/wd/225477', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '경기도 성남시 분당구 판교로 256번길 7, 넥슨', 'stack': ['데이터 분석']}
{'title': 'Junior Data Governance', 'company_name': '트릿지', 'detail_url': 'https://www.wanted.co.kr/wd/233338', 'platform_name': 'wanted', 'end_date': '2024.11.01', 'career': '경력 2-5년', 'region': '서울특별시 서초구 방배로 226', 'stack': ['기술 스택 없음']}
{'title': '[제품실] Product Manager', 'company_name': '한국신용데이터(KCD)', 'detail_url': 'https://www.wanted.co.kr/wd/228981', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7-20년', 'region': '서울특별시 강남구 테헤란로 127, 5층(하나금융그룹 강남사옥)', 'stack': ['기술 스택 없음']}
{'title': 'Data Scientist - Manufacturing Data (KR)', 'company_name': '가우스랩스(Gauss Labs)', 'detail_url': 'https://www.wanted.co.kr/wd/237781', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3년 이상', 'region': '서 울특별시 강남구 테헤란로 201', 'stack': ['Azure', 'Python', 'SQL', 'AWS']}
{'title': '데이터 분석가 (2년 이상)', 'company_name': '피엠그로우', 'detail_url': 'https://www.wanted.co.kr/wd/242335', 'platform_name': 'wanted', 'end_date': '2024.10.20', 'career': '경력 2년 이상', 'region': '서울시 서초구 반포대로 23길 6 효진빌딩 5층', 'stack': ['Pytorch', 'Tensorflow', 'SQL', '개발', '데이터 분석', 'EDA', 'ML']}
{'title': '빅데이터 개발자(전문연구요원가능)', 'company_name': '베스텔라랩', 'detail_url': 'https://www.wanted.co.kr/wd/230664', 'platform_name': 'wanted', 'end_date': '2024.10.21', 'career': '신입 이상', 'region': '안양시 동안구 엘에스로116번길 118 안양2차 SKV1 Center', 'stack': ['Git', 'React', 'Java', 'JavaScript', 'Python', 'Spring Framework']}
{'title': 'Data Analyst(3년 이상)', 'company_name': '버즈빌(buzzvil)', 'detail_url': 'https://www.wanted.co.kr/wd/243760', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울특별시 송파구 석촌호수로 272 MY빌딩 2~5층', 'stack': ['기술 스택 없음']}
{'title': '데이터파트 리더', 'company_name': '키즈노트', 'detail_url': 'https://www.wanted.co.kr/wd/233272', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7-15년', 'region': '경기도 성남시 분당구 판교역로 235 (H스퀘어, N동) 4층', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '엔미디어플랫폼', 'detail_url': 'https://www.wanted.co.kr/wd/233436', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년 이상', 'region': '서울시 송파구 법원로 114 A동 11층 ( 문정동, 엠스테이트)', 'stack': ['Zeplin', 'Linux', 'MongoDB', 'Python', 'SSIS', 'MSSQL']}
{'title': 'Data Analyst (Middle)', 'company_name': '와드(캐치테이블)', 'detail_url': 'https://www.wanted.co.kr/wd/222878', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 4-10년', 'region': '경기도 성남시 분당구 대왕판교 로 660 유스페이스1 A동 605호', 'stack': ['Python', 'SQL', 'Tableau']}
{'title': 'AI Engineer (전문연구요원 가능)', 'company_name': '라온데이터', 'detail_url': 'https://www.wanted.co.kr/wd/241115', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 2년', 'region': '서울특별시 장충단로 209 5층', 'stack': ['Pytorch', 'Python', 'Docker']}
{'title': '데이터 분석 엔진 개발  (6년이상)', 'company_name': '이글루코퍼레이션', 'detail_url': 'https://www.wanted.co.kr/wd/227199', 'platform_name': 'wanted', 'end_date': '2024.10.31', 'career': '경력 6-10년', 'region': '서울 송파구 문정 동 정의로 8길 7 한스빌딩 ㈜이글루코퍼레이션', 'stack': ['기술 스택 없음']}
{'title': '시계열 이상탐지 Data Scientist (전문연구요원 보충역 지원 가능)', 'company_name': '알티엠', 'detail_url': 'https://www.wanted.co.kr/wd/203738', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 1-3년', 'region': '서울시 강남구 역삼로3길 11 광성빌딩 10층', 'stack': ['Git', 'Linux', 'Pytorch', 'Scikit-Learn', 'Tensorflow', 'Python', 'OpenCV']}
{'title': 'Data Scientist', 'company_name': '앰버로드', 'detail_url': 'https://www.wanted.co.kr/wd/191938', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2년 이상', 'region': '서울 강남구 역삼동 선릉로 551', 'stack': ['기술 스택 없음']}
{'title': '데이터 분석가', 'company_name': '뮤즈라이브(Muzlive)', 'detail_url': 'https://www.wanted.co.kr/wd/222747', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-10년', 'region': '마포구 월드컵북로 56길 19, 드림타워 10층', 'stack': ['Data Analysis', 'Google Analytics']}
{'title': '데이터 비즈니스팀 팀장 (데이터 분석가)', 'company_name': '하쿠호도제일', 'detail_url': 'https://www.wanted.co.kr/wd/239678', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 6-10년', 'region': '마포구 마포대로 20 다보빌딩 8층', 'stack': ['기술 스택 없음']}
{'title': 'AI Solution - Data Manager', 'company_name': '업스테이지', 'detail_url': 'https://www.wanted.co.kr/wd/242072', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-15년', 'region': '리모트근무', 'stack': ['기술 스택 없음']}
{'title': 'Data Scientist (Product)', 'company_name': '토스뱅크', 'detail_url': 'https://www.wanted.co.kr/wd/203917', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-20년', 'region': '서울특별시 강남구 테헤란로 131', 'stack': ['모델링', '모델링 및 시뮬레이션', '머신 비전']}
{'title': 'ML Data Engineer (5년 이상)', 'company_name': '수퍼톤', 'detail_url': 'https://www.wanted.co.kr/wd/212794', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '강남구 학동로9길 13', 'stack': ['React', 'JavaScript', 'TypeScript']}
{'title': '데이터 기획 및 분석 (BI)', 'company_name': '해긴', 'detail_url': 'https://www.wanted.co.kr/wd/240082', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-10년', 'region': '구로구 경인로 662 디큐브시티 타워동 39 층', 'stack': ['기술 스택 없음']}
{'title': '빅데이터 분석 및 알고리즘 모델링 리더', 'company_name': '인튜브', 'detail_url': 'https://www.wanted.co.kr/wd/244533', 'platform_name': 'wanted', 'end_date': '2025.02.06', 'career': '경력 5년 이상', 'region': '강남구 학동로33길 17, 2층 (수산빌딩)', 'stack': ['Python', 'SQL', 'NoSQL', 'Restful API']}
{'title': '[2년 이상] 주니어 Software Engineer (Data)', 'company_name': '클럼엘', 'detail_url': 'https://www.wanted.co.kr/wd/239465', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-6년', 'region': '서울 강남구 테헤란로26길 12, 제일비전타워 2층', 'stack': ['기술 스택 없음']}
{'title': '미국 입시 에듀테크 Extracurricular Data Researcher', 'company_name': '링크12', 'detail_url': 'https://www.wanted.co.kr/wd/238954', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입', 'region': '강남구 신사동 523-24', 'stack': ['기술 스택 없음']}
{'title': '빅데이터 시스템 구축 및 운영 담당자(3년 이상)', 'company_name': '엑셈', 'detail_url': 'https://www.wanted.co.kr/wd/226657', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-7년', 'region': '서울 강서구 마곡중 앙8로5길 40 (마곡동) (주)엑셈', 'stack': ['Hadoop', 'Linux', 'Java', 'Tomcat', 'Shell Scripting']}
{'title': '[ETL/Crawling] Data Engineer', 'company_name': '디써클', 'detail_url': 'https://www.wanted.co.kr/wd/237885', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-9년', 'region': '서울시 마포구 마포대로 122 프론트 원', 'stack': ['Hadoop', 'Spark', 'Java', 'Python', 'Scala', 'GCP', 'Kubernetes']}
{'title': 'Data Pipeline Team Leader', 'company_name': '이마고웍스', 'detail_url': 'https://www.wanted.co.kr/wd/197658', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-10년', 'region': '서울시 강남구 강남대로 636', 'stack': ['Azure', 'Hadoop', 'Spark', 'Java', 'Python', 'SQL', 'Google Analytics', 'Power BI', 'Tableau', 'AWS', 'NoSQL', 'GCP']}
{'title': '[5년 이상] 데이터 엔지니어', 'company_name': '오아시스비즈니스', 'detail_url': 'https://www.wanted.co.kr/wd/231517', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년 이상', 'region': '서울시 강남구 선릉로 667, 5층', 'stack': ['MySQL', 'Python', 'SQL', 'Tableau', 'AWS', 'PostgreSQL', 'GCP']}
{'title': '[경영기획팀] IR Manager', 'company_name': '한국신용데이터(KCD)', 'detail_url': 'https://www.wanted.co.kr/wd/216547', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '서울특별시 강남구 테헤란 로 127, 5층(하나금융그룹 강남사옥)', 'stack': ['기술 스택 없음']}
{'title': '백엔드 개발자', 'company_name': '한국그린데이터', 'detail_url': 'https://www.wanted.co.kr/wd/222974', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 4-9년', 'region': '서초구 성촌길33, 삼성R&D캠퍼스 C동', 'stack': ['Git', 'Linux', 'MySQL', 'JavaScript', 'Python', 'AWS', 'Docker']}
{'title': 'Clinical Data Manager', 'company_name': '에이아이트릭스(AITRICS)', 'detail_url': 'https://www.wanted.co.kr/wd/242662', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2년 이상', 'region': '서울특별시 강남구 테 헤란로 218, AP Tower 13층', 'stack': ['기술 스택 없음']}
{'title': '데이터사이언티스트_교육팀', 'company_name': '데이콘', 'detail_url': 'https://www.wanted.co.kr/wd/244160', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 1년 이상', 'region': '서울특별시 영등포구 은행로 3 익스 콘벤처타워 901호', 'stack': ['기술 스택 없음']}
{'title': 'Data Engineer_운영 플랫폼 개발', 'company_name': '두나무(업비트/증권플러스)', 'detail_url': 'https://www.wanted.co.kr/wd/216576', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-20년', 'region': '서울시 서초 구 강남대로 369 (서초동, DF Tower)', 'stack': ['기술 스택 없음']}
{'title': 'Data Engineer', 'company_name': '데이터라이즈', 'detail_url': 'https://www.wanted.co.kr/wd/81834', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 1-10년', 'region': '서울특별시 강남구 도곡로 205, YK빌딩 6층', 'stack': ['Git', 'Java', 'JavaScript', 'Python', 'AWS', 'Docker', 'FastAPI']}
{'title': 'Software Engineer, Data Acquisition', 'company_name': '트릴리온랩스', 'detail_url': 'https://www.wanted.co.kr/wd/240910', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 10년', 'region': '삼성동 144-19', 'stack': ['Python', 'AWS']}
{'title': '[마켓팀] MD', 'company_name': '한국신용데이터(KCD)', 'detail_url': 'https://www.wanted.co.kr/wd/233495', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 6-10년', 'region': '서울특별시 강남구 테헤란로 127, 5층( 하나금융그룹 강남사옥)', 'stack': ['기술 스택 없음']}
{'title': '[전문연구요원 가능] ML Engineer', 'company_name': '데이터메이커', 'detail_url': 'https://www.wanted.co.kr/wd/195492', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3년 이상', 'region': '대전광역시 유성구 죽동 유성대로 871 CU빌딩 4층', 'stack': ['Git', 'Github', 'Linux', 'Python', 'Docker']}
{'title': 'Data Analyst', 'company_name': '토스뱅크', 'detail_url': 'https://www.wanted.co.kr/wd/195143', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-15년', 'region': '서울특별시 강남구 테헤란로 131', 'stack': ['기 술 스택 없음']}
{'title': 'Data Engineer(Jr)', 'company_name': '데이터라이즈', 'detail_url': 'https://www.wanted.co.kr/wd/226473', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 2년', 'region': '서울특별시 강남구 도곡로 205, YK빌딩 6층', 'stack': ['기술 스택 없음']}
{'title': '[경영기획팀] Corporate Development Manager', 'company_name': '한국신용데이터(KCD)', 'detail_url': 'https://www.wanted.co.kr/wd/223675', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-9년', 'region': '서울특 별시 강남구 테헤란로 127, 5층(하나금융그룹 강남사옥)', 'stack': ['기술 스택 없음']}
{'title': '빅데이터팀 리더 (데이터 사이언티스트)', 'company_name': '카운터포인트리서치', 'detail_url': 'https://www.wanted.co.kr/wd/242885', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 8년 이상', 'region': '서울시 강 남구 테헤란로19길 5, 6층', 'stack': ['MySQL', 'Python', 'SQL', 'AWS']}
{'title': 'Data Infra Platform Team Leader', 'company_name': '하이퍼커넥트', 'detail_url': 'https://www.wanted.co.kr/wd/241456', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '강남구 영동대로 517 아셈타워', 'stack': ['기술 스택 없음']}
{'title': '제품 Data Analyst (Middle)', 'company_name': '바비톡', 'detail_url': 'https://www.wanted.co.kr/wd/236633', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년 이상', 'region': '서울특별시 서초구 강남대로363, 363타워 11층', 'stack': ['기술 스택 없음']}
{'title': 'Data Engineer', 'company_name': '이마고웍스', 'detail_url': 'https://www.wanted.co.kr/wd/217492', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-5년', 'region': '서울시 강남구 강남대로 636', 'stack': ['Azure', 'Hadoop', 'Spark', 'Java', 'Python', 'SQL', 'Google Analytics', 'Power BI', 'Tableau', 'AWS', 'NoSQL', 'GCP']}       
{'title': '데이터사업본부 공공산업 영업대표', 'company_name': '크라우드웍스', 'detail_url': 'https://www.wanted.co.kr/wd/241180', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7-20년', 'region': '서울특별시 강남구 역삼 동 테헤란로 309 (삼성제일빌딩, 5층)', 'stack': ['기술 스택 없음']}
{'title': '데이터 사이언티스트', 'company_name': '브이피피랩', 'detail_url': 'https://www.wanted.co.kr/wd/240283', 'platform_name': 'wanted', 'end_date': '2024.10.31', 'career': '경력 5-10년', 'region': '서울시 강남구 역삼로 168, 팁스타운 S6 502호', 'stack': ['Git', 'Github', 'MySQL', 'Python', 'AWS']}
{'title': '데이터 사이언티스트', 'company_name': '위버스컴퍼니(WEVERSE COMPANY)', 'detail_url': 'https://www.wanted.co.kr/wd/225115', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-10년', 'region': '경기도 성남시 분당 구 분당내곡로 131 판교테크원 타워1, 6층', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '라플라스테크놀로지스', 'detail_url': 'https://www.wanted.co.kr/wd/156626', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-10년', 'region': '경기 성남시 분당구 정자일로 95  네이버 1784', 'stack': ['기술 스택 없음']}
{'title': '[SWING] 그로스 데이터 분석가', 'company_name': '더스윙', 'detail_url': 'https://www.wanted.co.kr/wd/241841', 'platform_name': 'wanted', 'end_date': '2024.11.30', 'career': '경력 3-10년', 'region': '용산구 서빙고로 17 센트럴파크타워', 'stack': ['기술 스택 없음']}
{'title': '[글로벌보안본부] 탐지솔루션실 데이터 애널리틱스 엔지니어', 'company_name': '넥슨코리아(NEXON)', 'detail_url': 'https://www.wanted.co.kr/wd/217823', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년 이상', 'region': '경기도 성남시 분당구 판교로 256번길 7, 넥슨', 'stack': ['React', 'C#', 'Java', 'JavaScript', 'Node.js', 'Python', 'DevOps', 'Bootstrap', 'Next.js']}
{'title': '이미지 Data Scientist 5년 이상(전문연구요원 보충역 지원 가능)', 'company_name': '알티엠', 'detail_url': 'https://www.wanted.co.kr/wd/180586', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-12년', 'region': '서울시 강남구 역삼로3길 11 광성빌딩 10층', 'stack': ['Git', 'Linux', 'Pytorch', 'Scikit-Learn', 'Tensorflow', 'Python', 'OpenCV']}
{'title': 'Data Analyst Team Leader (7년 이상)', 'company_name': '중고나라', 'detail_url': 'https://www.wanted.co.kr/wd/242165', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7년 이상', 'region': '서울시 서초구 서초대로301 동익성봉빌딩 7층', 'stack': ['Python', 'SQL']}
{'title': 'AI 의료 플랫폼 마케팅 (5년 이상)', 'company_name': '어반데이터랩', 'detail_url': 'https://www.wanted.co.kr/wd/244212', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-10년', 'region': '서울시 종로구 창경궁로 240-7 2층', 'stack': ['기술 스택 없음']}
{'title': '데이터 분석 담당자 (3년 이상)', 'company_name': '윌로그', 'detail_url': 'https://www.wanted.co.kr/wd/224170', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-6년', 'region': '서울특별시 강남구 삼성로 507, JS 타워 9층', 'stack': ['SQL', 'Data Analysis', '데이터 분석', '데이터 과학', 'Data Guard']}
{'title': 'Data Scientist', 'company_name': '에이비일팔공(AB180)', 'detail_url': 'https://www.wanted.co.kr/wd/197959', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 5년', 'region': '서울특별시 서초구 강남대로61길 17', 'stack': ['기술 스택 없음']}
{'title': '마케팅 Data Analyst (Middle)', 'company_name': '바비톡', 'detail_url': 'https://www.wanted.co.kr/wd/232133', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년 이상', 'region': '서울특별시 서초구 강남대로363, 363타워 11층', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '코인원(coinone)', 'detail_url': 'https://www.wanted.co.kr/wd/217308', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울특별시 영등포구 여의대로 108 파크원 타워1 45층, 46층', 'stack': ['기술 스택 없음']}
{'title': '기술영업팀 팀장급 PM(사업수행)', 'company_name': '한국그린데이터', 'detail_url': 'https://www.wanted.co.kr/wd/242397', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년 이상', 'region': '서초구 성촌길33, 삼성R&D캠퍼스 C동', 'stack': ['AutoCAD']}
{'title': '데이터분석컨설턴트', 'company_name': '콘센트릭스서비스코리아CATALYST', 'detail_url': 'https://www.wanted.co.kr/wd/228935', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-12년', 'region': '서울특별시 강남구  테헤란로 509, 엔씨타워 5층', 'stack': ['PowerPoint', 'EDA']}
{'title': 'AI/빅데이터 분야 B2G 기획 및 운영 담당자', 'company_name': '바이오에이아이', 'detail_url': 'https://www.wanted.co.kr/wd/235913', 'platform_name': 'wanted', 'end_date': '2024.10.30', 'career': '경력 3-20년', 'region': '서울 강남구 남부순환로 2621', 'stack': ['기술 스택 없음']}
{'title': 'Data Engineer(Data & Accounting)', 'company_name': '레브잇', 'detail_url': 'https://www.wanted.co.kr/wd/229645', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울 관악구 봉천로 456 영인 MC 6층', 'stack': ['MongoDB', 'Python']}
{'title': 'Data Scientist (10년 이상)', 'company_name': '디플래닉스(Dplanex)', 'detail_url': 'https://www.wanted.co.kr/wd/201396', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 10-15년', 'region': '서울특별시 서초구 강 남대로 465 교보타워 B동 4층', 'stack': ['BI', 'OLAP', 'ML']}
{'title': '데이터 사이언티스트(5년 이상)', 'company_name': '엑셈', 'detail_url': 'https://www.wanted.co.kr/wd/232305', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '서울 강서구 마곡중앙8로5길 40 (마 곡동) (주)엑셈', 'stack': ['Python', 'R', 'SQL', 'Tableau']}
{'title': '데이터 엔지니어', 'company_name': '강남언니(힐링페이퍼)', 'detail_url': 'https://www.wanted.co.kr/wd/195243', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3년 이상', 'region': '테헤란로 124, 삼원타워(변협 신회관 빌딩) 13층', 'stack': ['Spark', 'Java', 'Python', 'SQL', 'AWS']}
{'title': '[빗썸] 데이터 분석 개발자', 'company_name': '빗썸코리아', 'detail_url': 'https://www.wanted.co.kr/wd/214874', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 4-12년', 'region': '서울시 강남구 테헤란로 124, 삼원타워 14~18F', 'stack': ['React', 'TypeScript', 'Next.js']}
{'title': '신입 Software Engineer (Data)', 'company_name': '클럼엘', 'detail_url': 'https://www.wanted.co.kr/wd/239469', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입', 'region': '서울 강남구 테헤란로26길 12, 제일비전타워 2층', 'stack': ['기술 스택 없음']}
{'title': '전자서명/보안 소프트웨어 영업 총괄 팀장', 'company_name': '인텔렉추얼데이터', 'detail_url': 'https://www.wanted.co.kr/wd/231361', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년 이상', 'region': '서울시 강 남구 역삼로25길 37, 1층', 'stack': ['기술 스택 없음']}
{'title': '지도 데이터 입력', 'company_name': '베스텔라랩', 'detail_url': 'https://www.wanted.co.kr/wd/238561', 'platform_name': 'wanted', 'end_date': '2024.10.21', 'career': '신입', 'region': '안양시 동안구 엘에스로116번길 118 안양2차 SKV1 Center', 'stack': ['기술 스택 없음']}
{'title': '블록체인 온체인 데이터 엔지니어', 'company_name': '미르니', 'detail_url': 'https://www.wanted.co.kr/wd/240918', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 4년', 'region': '서울시 관악로 12길 10', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '셰어라운드', 'detail_url': 'https://www.wanted.co.kr/wd/161537', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-10년', 'region': '서울특별시 종로구 종로 6, 광화문우체국 5층 셰어라운드(주) / 5호선 광화문역 1분 거리', 'stack': ['iOS', 'Kotlin', 'Python', 'Scala', 'SQL', 'AWS', 'Spring Framework', 'GCP', 'React.js', 'React Native']}
{'title': '마케팅 데이터 분석 (브랜드엑셀러레이팅사업부/데이터분석팀)', 'company_name': '테크랩스', 'detail_url': 'https://www.wanted.co.kr/wd/239888', 'platform_name': 'wanted', 'end_date': '2024.10.31', 'career': '경력 2-10년', 'region': '서울특별시 강남구 강남대로84길 13 (역삼동,KR타워) 5층,6층,7층,8층,9층', 'stack': ['기술 스택 없음']}
{'title': '글로벌 마케팅 데이터 분석가', 'company_name': '아티언스', 'detail_url': 'https://www.wanted.co.kr/wd/239098', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-4년', 'region': '마포구 양화로 45 세아타워 12층', 'stack': ['SQL', 'Excel', 'Google Analytics', 'Tableau', 'Adobe', 'GCP']}
{'title': '쿠팡 Manager, Data Analyst (Coupang Advertising)', 'company_name': '쿠팡', 'detail_url': 'https://www.wanted.co.kr/wd/241286', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년 이상', 'region': '서울시 송파구 신천동 7-25 (월드타워)', 'stack': ['Python', 'SQL']}
{'title': '[그루비 SaaS 솔루션] 데이터 분석가', 'company_name': '플래티어', 'detail_url': 'https://www.wanted.co.kr/wd/177463', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입', 'region': '서울시 송파구 법원로9길 26  H Business Park D동 6층', 'stack': ['기술 스택 없음']}
{'title': '실시간 데이터 통신 및 데이터 처리 개발자', 'company_name': '아이오티플렉스', 'detail_url': 'https://www.wanted.co.kr/wd/225573', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 10년', 'region': '부산 해운 대구 센텀동로 99, 벽산이센텀클래스원 601호', 'stack': ['Github', 'MySQL', 'React', 'CSS', 'HTML', 'Java', 'JavaScript', 'Python', 'SQL', 'CSS3', 'Spring Framework', 'Spring Boot']}
{'title': '[쿠팡] Manager, Data Analyst (Coupang Ad)', 'company_name': '쿠팡', 'detail_url': 'https://www.wanted.co.kr/wd/241018', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7년 이상', 'region': '선릉역 부근 HJ타워', 'stack': ['Python', 'R', 'SQL', 'Tableau']}
{'title': 'Data Analytics Engineer', 'company_name': '데이터라이즈', 'detail_url': 'https://www.wanted.co.kr/wd/177620', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-10년', 'region': '서울특별시 강남구 도곡로 205, YK빌딩 6층', 'stack': ['Git', 'Java', 'JavaScript', 'Python', 'AWS', 'Docker', 'FastAPI']}
{'title': 'Data Governance Manager', 'company_name': '트릿지', 'detail_url': 'https://www.wanted.co.kr/wd/227242', 'platform_name': 'wanted', 'end_date': '2024.11.01', 'career': '경력 5-10년', 'region': '서울특별시 서초구 방배로 226', 'stack': ['Git', 'Python', 'Google Analytics', 'AWS', 'Spring Framework', 'Amplitude']}
{'title': '[내집스캔] 개발 리드 팀원', 'company_name': '한국부동산데이터연구소', 'detail_url': 'https://www.wanted.co.kr/wd/190361', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '서울특별사 마포구 마포대로 122, 17층', 'stack': ['Android', 'iOS', 'Linux', 'MySQL', 'React', 'VueJS', 'HTML', 'JavaScript', 'PHP', 'Python', 'AWS', 'Docker', 'Flutter']}
{'title': 'Data Analyst', 'company_name': '클로버추얼패션(CLOVirtualFashion)', 'detail_url': 'https://www.wanted.co.kr/wd/210507', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울 강남구 테헤란로 152', 'stack': ['Slack', 'SQL', 'BI', 'Tableau']}
{'title': '[5년 이상] 데이터 사이언티스트', 'company_name': '오아시스비즈니스', 'detail_url': 'https://www.wanted.co.kr/wd/231691', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년 이상', 'region': '서울시 강남구 선릉 로 667, 5층', 'stack': ['Git', 'MySQL', 'Pytorch', 'Tensorflow', 'Python', 'SQL', 'Docker', 'GCP']}
{'title': '[빗썸] 비즈니스 데이터 분석', 'company_name': '빗썸코리아', 'detail_url': 'https://www.wanted.co.kr/wd/209268', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7-10년', 'region': '서울시 강남구 테헤란로 124, 삼원타워 14~18F', 'stack': ['기술 스택 없음']}
{'title': '[글로벌디지털마케팅] 데이터 엔지니어', 'company_name': '콘센트릭스서비스코리아CATALYST', 'detail_url': 'https://www.wanted.co.kr/wd/96707', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 8-13년', 'region': '영등포구 여의도동', 'stack': ['기술 스택 없음']}
{'title': 'Database Engineer (DBE)', 'company_name': '카카오페이증권(kakaopay증권)', 'detail_url': 'https://www.wanted.co.kr/wd/238253', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년 이상', 'region': '경기도 성남시 분당구 판교역로 166 카카오아지트', 'stack': ['MongoDB', 'MySQL']}
{'title': 'IMC마케팅팀장(퍼포먼스마케터 및 데이터분석가)', 'company_name': '아이트럭', 'detail_url': 'https://www.wanted.co.kr/wd/234598', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 9-12년', 'region': '서울특별시 서 초구 동작대로 230, 10층(방배동,화련회관)', 'stack': ['기술 스택 없음']}
{'title': '[쿠팡] Data Analyst', 'company_name': '쿠팡', 'detail_url': 'https://www.wanted.co.kr/wd/240450', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년 이상', 'region': '서울특별시 송파구 송파대로 570', 'stack': ['SQL', 'Excel', 'Tableau']}
{'title': 'Big Data Engineer', 'company_name': '현대캐피탈', 'detail_url': 'https://www.wanted.co.kr/wd/233632', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 4-10년', 'region': '현대캐피탈 본사(서울 중구 세종대로 14 그랜드센트럴 A동)', 'stack': ['Hadoop', 'Python', 'AWS', 'Kubernetes']}
{'title': 'Data PM (Governance)', 'company_name': '트릿지', 'detail_url': 'https://www.wanted.co.kr/wd/238777', 'platform_name': 'wanted', 'end_date': '2024.11.01', 'career': '경력 5-10년', 'region': '서울특별시 서초구 방배로 226', 'stack': ['기술 스택 없음']}
{'title': 'IT기획 담당자', 'company_name': '헥토데이터', 'detail_url': 'https://www.wanted.co.kr/wd/231505', 'platform_name': 'wanted', 'end_date': '2024.10.16', 'career': '경력 5-15년', 'region': '서울특별시 강남구 테헤란로 223, 큰길타워 5층', 'stack': ['기술 스택 없음']}
{'title': 'Global Contents Marketing Manager (3년 이상)', 'company_name': '데이터뱅크', 'detail_url': 'https://www.wanted.co.kr/wd/221427', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-5년', 'region': '강남구 테헤란 로86길 12, 2층', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '카카오페이증권(kakaopay증권)', 'detail_url': 'https://www.wanted.co.kr/wd/227528', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7-15년', 'region': '경기도 성남시 분당구 판 교역로 166 카카오아지트', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '하이퍼리즘', 'detail_url': 'https://www.wanted.co.kr/wd/229693', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 15년', 'region': '서울 관악구 관악로 116 2층', 'stack': ['기술 스택 없음']}
{'title': '데이터 분석 엔지니어', 'company_name': '펫닥', 'detail_url': 'https://www.wanted.co.kr/wd/216979', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-7년', 'region': '서울 서초구 강남대로79길 32, 5층', 'stack': ['Python', 'AWS', 'NoSQL', 'Docker', 'RDBMS']}
{'title': 'Data Scientist Part Leader', 'company_name': '코그넷나인', 'detail_url': 'https://www.wanted.co.kr/wd/239757', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 8-15년', 'region': '서울시 서초구 강남대로351 청남 빌딩 9층', 'stack': ['기술 스택 없음']}
{'title': '[빗썸] 데이터분석팀 팀장', 'company_name': '빗썸코리아', 'detail_url': 'https://www.wanted.co.kr/wd/227704', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 8-12년', 'region': '서울시 강남구 테헤란로 124, 삼원 타워 14~18F', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '워트인텔리전스', 'detail_url': 'https://www.wanted.co.kr/wd/26823', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서초구 강남대로 507, 6층', 'stack': ['Git', 'Hadoop', 'Linux', 'MySQL', 'Redis', 'Spark', 'Java', 'Kotlin', 'Python', 'AWS', 'NoSQL', 'Docker', 'RDBMS', 'PostgreSQL', 'HBase']}
{'title': 'Sensor Data Verification Engineer', 'company_name': '모라이(MORAI)', 'detail_url': 'https://www.wanted.co.kr/wd/217712', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 15년', 'region': '서울시 강남구 테헤란로 78길 16, 7층', 'stack': ['Git', 'Python', 'C++', 'CAN', 'UDP', 'OpenCV', 'LiDAR', 'ROS']}
{'title': '분석 통계 데이터 시스템 개발자', 'company_name': '골든플래닛', 'detail_url': 'https://www.wanted.co.kr/wd/241026', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 10-14년', 'region': '강남구 언주로 535, 골든플 래닛 타워', 'stack': ['MySQL', 'JavaScript', 'Python', 'SQL']}
{'title': '[Data 관련] PM(Project Manager)', 'company_name': '인포유앤컴퍼니', 'detail_url': 'https://www.wanted.co.kr/wd/242621', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-10년', 'region': '서울시 성동구 성수일로 111, 선명스퀘어 311호', 'stack': ['Azure', 'BI', 'Power BI', 'Tableau', 'AWS']}
{'title': '[인텔리전스랩스] 게임밸류에이션팀 데이터 사이언티스트', 'company_name': '넥슨코리아(NEXON)', 'detail_url': 'https://www.wanted.co.kr/wd/230564', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '경기도 성남시 분당구 판교로 256번길 7, 넥슨', 'stack': ['Spark', 'Python', 'R', 'SQL', 'ML']}
{'title': '세일즈포스 CRM 데이터 활용,프로젝트 컨설턴트 PL', 'company_name': '골든플래닛', 'detail_url': 'https://www.wanted.co.kr/wd/236623', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '강남구 언 주로 535, 골든플래닛 타워', 'stack': ['Salesforce.com', 'CRM', 'CRM 소프트웨어', 'CRM 데이터베이스', 'CRM 통합']}       
{'title': 'Lead Data Engineer, ML Data Platform', 'company_name': '트웰브랩스(Twelve Labs)', 'detail_url': 'https://www.wanted.co.kr/wd/222696', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '서울시  용산구 이태원로 27길 39-11', 'stack': ['Hadoop', 'Pytorch', 'Tensorflow', 'Python']}
{'title': 'DBA·데이터엔지니어링', 'company_name': '다날(Danal)', 'detail_url': 'https://www.wanted.co.kr/wd/244850', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 10년 이상', 'region': '성남 분당구 분당로 55, 퍼스트타워 9층', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '핵클(Hackle)', 'detail_url': 'https://www.wanted.co.kr/wd/159253', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-20년', 'region': '서울시 강남구 논현로75길 8, 비드빌딩 2층', 'stack': ['Spark', 'Java', 'Kotlin', 'AWS', 'ETL', 'Spring Framework', 'JPA', 'Kubernetes']}
{'title': '빅데이터/알고리즘 개발자', 'company_name': '에스디바이오센서', 'detail_url': 'https://www.wanted.co.kr/wd/225593', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-13년', 'region': '서울특별시 강남구 도산대로 509', 'stack': ['기술 스택 없음']}
{'title': '데이터 연구 개발 R&D', 'company_name': '휴머스온', 'detail_url': 'https://www.wanted.co.kr/wd/241264', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5년', 'region': '서울특별시 강남구 선릉로 433, 10층', 'stack': ['기술 스택 없음']}
{'title': 'Onchain Data Analyst (퀀트 리서처)', 'company_name': '하이퍼리즘', 'detail_url': 'https://www.wanted.co.kr/wd/215345', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입', 'region': '서울 관악구 관악로 116 2층', 'stack': ['Git', 'Python', 'TypeScript', 'Docker']}
{'title': 'AI 학습데이터 프로젝트 매니저(PM)', 'company_name': '셀렉트스타', 'detail_url': 'https://www.wanted.co.kr/wd/57391', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 3년', 'region': '서울특별시 강남구 테헤 란로 20길 20', 'stack': ['기술 스택 없음']}
{'title': '데이터 분석가(DA) (2년차 이상)', 'company_name': '와이피랩스(커넥팅)', 'detail_url': 'https://www.wanted.co.kr/wd/220545', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-15년', 'region': '서울특별시 강남구  영동대로 424 3층', 'stack': ['기술 스택 없음']}
{'title': '빅데이터 플랫폼 BI 프로젝트 관리', 'company_name': '펑타이그레이터차이나(피티코리아)', 'detail_url': 'https://www.wanted.co.kr/wd/238282', 'platform_name': 'wanted', 'end_date': '2024.10.27', 'career': '경력 3-11년', 'region': '(06241) 서울 강남구 강남대로 372 FINE TOWER - 신분당선 강남 에서 200m 이내', 'stack': ['기술 스택 없음']}
{'title': '이미지 Data Scientist (주니어, 전문연구요원 보충역 지원 가능)', 'company_name': '알티엠', 'detail_url': 'https://www.wanted.co.kr/wd/207150', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 1-3년', 'region': ' 서울시 강남구 역삼로3길 11 광성빌딩 10층', 'stack': ['기술 스택 없음']}
{'title': '콘텐츠 프로듀서 [데이터 사이언스 교육]', 'company_name': '코드잇(codeit)', 'detail_url': 'https://www.wanted.co.kr/wd/218263', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 10년', 'region': '중구 청계천 로 100 시그니쳐타워 동관 10층', 'stack': ['Python', 'SQL', '딥 러닝', 'Excel', 'Tableau', '데이터 분석', '데이터 과학']}
{'title': 'Data Pipeline Engineer', 'company_name': '두나무(업비트/증권플러스)', 'detail_url': 'https://www.wanted.co.kr/wd/239395', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '서울시 서초구 강남대로 369 (서초동, DF Tower)', 'stack': ['기술 스택 없음']}
{'title': 'Data Scientist', 'company_name': '한국그린데이터', 'detail_url': 'https://www.wanted.co.kr/wd/210157', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-7년', 'region': '서초구 성촌길33, 삼성R&D캠퍼스 C동', 'stack': ['Git', 'Github', 'MongoDB', 'MySQL', 'React', 'JavaScript', 'Node.js', 'TypeScript', 'AWS', 'Docker']}
{'title': '데이터분석가', 'company_name': '넛지헬스케어(캐시워크)', 'detail_url': 'https://www.wanted.co.kr/wd/157552', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 3년', 'region': '서울특별시 강남구 역삼로1길 8', 'stack': ['기술 스택 없음']}
{'title': 'Data Engineer', 'company_name': '모플', 'detail_url': 'https://www.wanted.co.kr/wd/154911', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 1-3년', 'region': '대전팁스타운(충남대학교)', 'stack': ['Git', 'MongoDB', 'Pytorch', 'Spark', 'Node.js', 'Python', 'RDBMS', 'Kubernetes']}
{'title': 'Data Engineer(Lead)', 'company_name': '쏘카(SOCAR)', 'detail_url': 'https://www.wanted.co.kr/wd/234456', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-10년', 'region': '서울특별시 성동구 왕십리로 83-21, 아 크로서울포레스트 디타워 3-5F', 'stack': ['기술 스택 없음']}
{'title': '전기공사 공무 엔지니어(사업수행 및 지원업무)', 'company_name': '한국그린데이터', 'detail_url': 'https://www.wanted.co.kr/wd/227296', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 1-15년', 'region': '서초구 성촌길33, 삼성R&D캠퍼스 C동', 'stack': ['Excel', 'AutoCAD']}
{'title': '[Photoism] Product Data Analyst', 'company_name': '서북', 'detail_url': 'https://www.wanted.co.kr/wd/237939', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 6-9년', 'region': '학동로 224, 삼환아르누보3차 2층', 'stack': ['기술 스택 없음']}
{'title': '[장애인전형] 데이터 플랫폼 엔지니어', 'company_name': '콘센트릭스서비스코리아CATALYST', 'detail_url': 'https://www.wanted.co.kr/wd/242874', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-13년', 'region': '서울특별시 강남구 테헤란로 509, 엔씨타워 5층', 'stack': ['기술 스택 없음']}
{'title': '제품 Data Analyst (Senior)', 'company_name': '바비톡', 'detail_url': 'https://www.wanted.co.kr/wd/236632', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7년 이상', 'region': '서울특별시 서초구 강남대로363, 363타워 11층', 'stack': ['기술 스택 없음']}
{'title': '[내집스캔] 프로덕트 디자이너 팀원', 'company_name': '한국부동산데이터연구소', 'detail_url': 'https://www.wanted.co.kr/wd/221895', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울시 마포 구 백범로 31길 21', 'stack': ['서비스 디자인', '웹 디자인', 'UI 디자인', 'Figma']}
{'title': '[6년 이상] 시니어 Software Engineer (Data)', 'company_name': '클럼엘', 'detail_url': 'https://www.wanted.co.kr/wd/239467', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 6-15년', 'region': '서울 강남구 테헤란 로26길 12, 제일비전타워 2층', 'stack': ['기술 스택 없음']}
{'title': 'Data Analyst', 'company_name': '엘박스', 'detail_url': 'https://www.wanted.co.kr/wd/231121', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 6년 이상', 'region': '서울특별시 강남구 테헤란로 306, 카이트타워 5층', 'stack': ['Amplitude']}
{'title': 'Database Reliability Engineer, DBRE', 'company_name': '두나무(업비트/증권플러스)', 'detail_url': 'https://www.wanted.co.kr/wd/236606', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '서울시 서초구 강남대로 369 (서초동, DF Tower)', 'stack': ['기술 스택 없음']}
{'title': '데이터베이스 관리자(DBA)', 'company_name': '아이스크림에듀', 'detail_url': 'https://www.wanted.co.kr/wd/177307', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 10-17년', 'region': '서울특별시 서초구 매헌로 16, 하이브랜드빌딩 16층/ 18층', 'stack': ['Pytorch', 'Tensorflow']}
{'title': '시계열 이상탐지 Data Scientist 5년 이상(전문연 보충역 지원 가능)', 'company_name': '알티엠', 'detail_url': 'https://www.wanted.co.kr/wd/176470', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-12년', 'region': '서울시 강남구 역삼로3길 11 광성빌딩 10층', 'stack': ['Git', 'Linux', 'Pytorch', 'Scikit-Learn', 'Tensorflow', 'Python', 'OpenCV']}
{'title': '데이터 엔지니어 (일본)', 'company_name': '딥세일즈', 'detail_url': 'https://www.wanted.co.kr/wd/230461', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 1년 이상', 'region': '강남구 대치동 944-21, 3층', 'stack': ['Hadoop', 'Spark', 'Python', 'SQL', 'SQL 서버', 'API', 'API 개발']}
{'title': '채권 관리 빅데이터/AI 플랫폼 CEO 직속 비서(CEO Staff)', 'company_name': '채권마루', 'detail_url': 'https://www.wanted.co.kr/wd/240523', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 5년', 'region': '서울특별시 강남구 도곡로 218(CNCITY Tower), 4층 4호', 'stack': ['비서 기술']}
{'title': 'Database Administrator(DBA)', 'company_name': '인터파크트리플', 'detail_url': 'https://www.wanted.co.kr/wd/230597', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7년 이상', 'region': '서울시 서초구 강남대로 447 남서울빌딩', 'stack': ['기술 스택 없음']}
{'title': '[하나금융융합기술원] Data Scientist', 'company_name': '하나금융티아이', 'detail_url': 'https://www.wanted.co.kr/wd/145246', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '서울특별시 강남구 테헤란로 127', 'stack': ['Confluence', 'GitLab', 'Slack', 'Pytorch', 'Python', 'OpenCV']}
{'title': '데이터분석가 (병역특례_현역/보충역)', 'company_name': '넛지헬스케어(캐시워크)', 'detail_url': 'https://www.wanted.co.kr/wd/180914', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입', 'region': '서울특별시 강남구 역삼로1길 8', 'stack': ['기술 스택 없음']}
{'title': '[인텔리전스랩스] 데이터 분석가 (사운드 AI 연구)', 'company_name': '넥슨코리아(NEXON)', 'detail_url': 'https://www.wanted.co.kr/wd/239839', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-35년', 'region': '경 기도 성남시 분당구 판교로 256번길 7, 넥슨', 'stack': ['Pytorch', 'Python', 'ML']}
{'title': 'Data Scientist', 'company_name': '바이온사이트', 'detail_url': 'https://www.wanted.co.kr/wd/167495', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '성동구 왕십리로58 (FORHU) 418호', 'stack': ['Git', 'Pytorch', 'React', 'Tensorflow', 'Python', 'R', 'Docker', 'Notion']}
{'title': '데이터 분석가', 'company_name': '카카오페이증권(kakaopay증권)', 'detail_url': 'https://www.wanted.co.kr/wd/221127', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7-15년', 'region': '경기도 성남시 분당구 판교 역로 166 카카오아지트', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '쉬모스랩', 'detail_url': 'https://www.wanted.co.kr/wd/226308', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-6년', 'region': '서울 강남구 논현로26길 21', 'stack': ['Linux', 'C / C++', 'JavaScript', 'Python', 'SQL']}
{'title': '[산업기능요원(병역특례, 보충역)] 데이터 엔지니어', 'company_name': '와이피랩스(커넥팅)', 'detail_url': 'https://www.wanted.co.kr/wd/219655', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입', 'region': '서울특별시 강남구 영동대로 424 3층', 'stack': ['기술 스택 없음']}
{'title': '데이터 분석 담당자 리드 (10년 이상)', 'company_name': '윌로그', 'detail_url': 'https://www.wanted.co.kr/wd/223258', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 10-15년', 'region': '서울특별시 강남구 삼성로 507, JS타워 9층', 'stack': ['Python', 'R', 'Data Analysis', 'BI', 'Tableau', '데이터베이스', '데이터 분석', '데이터 구조', '데이터 과학', '데이터 통합', '데이터 수집', 'ML']}
{'title': '데이터 엔지니어 리더', 'company_name': '숲(SOOP)', 'detail_url': 'https://www.wanted.co.kr/wd/231165', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 10년', 'region': '경기도 성남시 분당구 삼평동 625 판교세븐 밴처밸리 1단지 2동', 'stack': ['Java', 'Python']}
{'title': '[산업기능요원(병역특례, 보충역)] 데이터 분석가(DA)', 'company_name': '와이피랩스(커넥팅)', 'detail_url': 'https://www.wanted.co.kr/wd/220540', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-15년', 'region': '서울특별시 강남구 영동대로 424 3층', 'stack': ['기술 스택 없음']}
{'title': 'Sr-Staff, Data Analyst(eCommerce Product)', 'company_name': '쿠팡', 'detail_url': 'https://www.wanted.co.kr/wd/223481', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-17년', 'region': '서울특별시 송파구 송파대로 570', 'stack': ['Python', 'R', 'SQL']}
{'title': 'Data Analyst', 'company_name': '부릉', 'detail_url': 'https://www.wanted.co.kr/wd/204725', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-12년', 'region': '서울특별시 서초구 강남대로 577 HY빌딩 3층/4층', 'stack': ['기술 스택 없음']}
{'title': '데이터 엔지니어', 'company_name': '씨제이이엔엠(CJ ENM)', 'detail_url': 'https://www.wanted.co.kr/wd/220866', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 8-15년', 'region': '서울시 마포구 상암산로66 CJ ENM 센터', 'stack': ['기술 스택 없음']}
{'title': '데이터 분석가', 'company_name': '브릭', 'detail_url': 'https://www.wanted.co.kr/wd/206929', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '경기도 성남시 분당구 판교역로 230', 'stack': ['기 술 스택 없음']}
{'title': '빅데이터 처리 시스템 개발', 'company_name': '파수', 'detail_url': 'https://www.wanted.co.kr/wd/216863', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울특별시 마포구 월드컵북로 396 누리 꿈스퀘어 비즈니스타워 6층/17층', 'stack': ['Hadoop', 'MySQL', 'Oracle', 'Spark', 'Java', 'JavaScript', 'Python', 'Swift', 'C', 'C++', 'jQuery', 'HTML5', 'HP', 'Spring Framework', 'MFC']}
{'title': '빅데이터 솔루션 (Elastic) 데이터 엔지니어', 'company_name': '위앤유텍', 'detail_url': 'https://www.wanted.co.kr/wd/245636', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-12년', 'region': '송파구 백제고분로 7길 7, 12층', 'stack': ['Linux', 'JavaScript', 'Python', 'SQL', 'AWS', 'Spring Framework']}
{'title': '언어 데이터 사업 해외영업 담당', 'company_name': '플리토', 'detail_url': 'https://www.wanted.co.kr/wd/245612', 'platform_name': 'wanted', 'end_date': '2024.10.28', 'career': '경력 1-5년', 'region': '서울특별시 강남구 영동대로 96 길 20 대화빌딩, 6층', 'stack': ['기술 스택 없음']}
{'title': '프로덕트 기획자(PM/PO)', 'company_name': '데이터메이커', 'detail_url': 'https://www.wanted.co.kr/wd/216485', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-20년', 'region': '대전광역시 유성구 죽동 유성대로 871 CU빌딩 4층', 'stack': ['Adobe XD', 'Figma']}
{'title': '빅데이터 솔루션 데이터 엔지니어 (5~13년)', 'company_name': '로이드케이', 'detail_url': 'https://www.wanted.co.kr/wd/245430', 'platform_name': 'wanted', 'end_date': '2024.11.17', 'career': '경력 5-13년', 'region': '경기도 성남시', 'stack': ['기술 스택 없음']}
{'title': '바이오 기업 데이터베이스 생성 및 관리', 'company_name': '바이오북', 'detail_url': 'https://www.wanted.co.kr/wd/244685', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입', 'region': '서울특별시 성동구', 'stack': ['기술 스택 없음']}
{'title': '보안 솔루션 기술 지원 엔지니어 (5년 이하 경력) - 판교', 'company_name': '지란지교데이터', 'detail_url': 'https://www.wanted.co.kr/wd/245025', 'platform_name': 'wanted', 'end_date': '2024.10.31', 'career': '경력 1-5년', 'region': '경기도 성남시 수정구 금토로 80번길 37 인피니티타워 W동 10층', 'stack': ['Linux', 'Windows 서버', 'IT 운영', 'CentOS', '보안 운영']}
{'title': '데이터베이스 관리자(DBA)', 'company_name': '핀업', 'detail_url': 'https://www.wanted.co.kr/wd/245005', 'platform_name': 'wanted', 'end_date': '2024.11.10', 'career': '경력 5-7년', 'region': '서울시 강남구 삼성로95길 10, 2-4층', 'stack': ['기술 스택 없음']}
{'title': '데이터베이스 관리자', 'company_name': '서플러스글로벌', 'detail_url': 'https://www.wanted.co.kr/wd/244972', 'platform_name': 'wanted', 'end_date': '2024.10.29', 'career': '경력 5-10년', 'region': '용인시 처인구 남사읍 서촌로 56', 'stack': ['PostgreSQL', 'MSSQL']}
{'title': '의약품 시장 데이터 사업(BD)', 'company_name': '이노케어플러스', 'detail_url': 'https://www.wanted.co.kr/wd/244198', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3년 이상', 'region': '서울특별시 서초구 효령로 107, 4층(방배동)', 'stack': ['기술 스택 없음']}
{'title': 'SAS 기반 데이터처리 전문가(7년 이상)', 'company_name': '알테어엔지니어링', 'detail_url': 'https://www.wanted.co.kr/wd/242909', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 7년 이상', 'region': '경기도 성남시 분당구 황새울로 216, 6층 알테어', 'stack': ['React', 'SAS']}
{'title': '[장애인전형] 데이터분석컨설턴트', 'company_name': '콘센트릭스서비스코리아CATALYST', 'detail_url': 'https://www.wanted.co.kr/wd/242870', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-12년', 'region': '서울특별시 강남구 테헤란로 509, 엔씨타워 5층', 'stack': ['기술 스택 없음']}
{'title': 'Head of Credit Risk and Data', 'company_name': '에잇퍼센트(8PERCENT)', 'detail_url': 'https://www.wanted.co.kr/wd/230893', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 10-20년', 'region': '서울특별시 영등포 구 국제금융로2길 17, 13층', 'stack': ['Python']}
{'title': 'Database Architect (DBA)', 'company_name': '인핸스(Enhans)', 'detail_url': 'https://www.wanted.co.kr/wd/154307', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 4-28년', 'region': '서울특별시 서초구 서초대로48 길 61 (1~4층)', 'stack': ['React', 'CSS', 'HTML', 'Swift', 'TypeScript', 'UX', 'Redux', 'Next.js']}
{'title': 'DevOps 개발자', 'company_name': '데이터메이커', 'detail_url': 'https://www.wanted.co.kr/wd/216073', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-20년', 'region': '대전광역시 유성구 죽동 유성대로 871 CU빌딩 4층', 'stack': ['Adobe XD', 'Figma']}
{'title': '웹 그래픽(F/E) 개발자', 'company_name': '데이터메이커', 'detail_url': 'https://www.wanted.co.kr/wd/216081', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-20년', 'region': '대전광역시 유성구 죽동 유성대로 871 CU빌딩 4층', 'stack': ['Adobe XD', 'Figma']}
{'title': '프론트엔드 개발자', 'company_name': '데이터메이커', 'detail_url': 'https://www.wanted.co.kr/wd/195494', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-20년', 'region': '대전광역시 유성구 죽동 유성대로 871 CU빌딩 4층', 'stack': ['Adobe XD', 'Figma']}
{'title': '백엔드 개발자', 'company_name': '데이터메이커', 'detail_url': 'https://www.wanted.co.kr/wd/195493', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-20년', 'region': '대전광역시 유성구 죽동 유성대로 871 CU빌딩 4층', 'stack': ['Adobe XD', 'Figma']}
{'title': '서버 및 인프라 개발자', 'company_name': '데이터메이커', 'detail_url': 'https://www.wanted.co.kr/wd/216068', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-20년', 'region': '대전광역시 유성구 죽동 유성대로 871 CU빌딩 4층', 'stack': ['Adobe XD', 'Figma']}
{'title': '데이터 구축 및 운영', 'company_name': '에비드넷', 'detail_url': 'https://www.wanted.co.kr/wd/191630', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울특별시 강남구 강남대로 292, 3층(뱅뱅빌딩)', 'stack': ['기술 스택 없음']}
{'title': 'Senior/Staff Data Scientist - Growth Analytics', 'company_name': '몰로코', 'detail_url': 'https://www.wanted.co.kr/wd/189666', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-20년', 'region': '서울 강남구 역 삼 센터필드 West 8층', 'stack': ['기술 스택 없음']}
{'title': 'Data Scientist in Financial', 'company_name': '페니로이스', 'detail_url': 'https://www.wanted.co.kr/wd/213503', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-10년', 'region': '서울특별시 종로구 율곡로2길 7 서머셋팰리스서울 4층', 'stack': ['Python', 'R']}
{'title': '시니어 데이터 엔지니어', 'company_name': '교보문고', 'detail_url': 'https://www.wanted.co.kr/wd/168560', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 8-15년', 'region': '서울특별시 마포구 월드컵북로54길 25 (상암동), DMC푸르지오시티 7층 교보문고 디지털혁신지원실', 'stack': ['기술 스택 없음']}
{'title': '특허 빅데이터분석가 (IP빅데이터분석팀)', 'company_name': '워트인텔리전스', 'detail_url': 'https://www.wanted.co.kr/wd/130444', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-15년', 'region': '서초구 강남대로 507, 6층', 'stack': ['IP', '데이터 분석', '특허', '특허 전략', '특허 분석']}
{'title': 'Data Quality Specialist [데이터팀]', 'company_name': '원티드랩', 'detail_url': 'https://www.wanted.co.kr/wd/153864', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-7년', 'region': '송파구 올림픽로 300 롯데월드타워 35층', 'stack': ['기술 스택 없음']}
{'title': '[Coupang] Staff, Data Analyst (Eats Strategy)', 'company_name': '쿠팡', 'detail_url': 'https://www.wanted.co.kr/wd/196128', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 6-20년', 'region': '강남구 테헤란로 332 HJ 타워', 'stack': ['Python', 'SQL', 'Product Management']}
{'title': 'Senior Data Scientist - Ads Performance', 'company_name': '몰로코', 'detail_url': 'https://www.wanted.co.kr/wd/191088', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-20년', 'region': '서울 강남구 역삼 센터 필드 West 8층', 'stack': ['기술 스택 없음']}
{'title': 'Data Engineer', 'company_name': '인핸스(Enhans)', 'detail_url': 'https://www.wanted.co.kr/wd/154545', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-8년', 'region': '서울특별시 서초구 서초대로48길 61 (1~4층)', 'stack': ['React', 'CSS', 'HTML', 'Swift', 'TypeScript', 'UX', 'Redux', 'Next.js']}
{'title': '데이터 엔지니어', 'company_name': '교보문고', 'detail_url': 'https://www.wanted.co.kr/wd/84863', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-15년', 'region': '서울특별시 마포구 월드컵북로54길 25 (상암동), DMC푸르지오시티 7층 교보문고 디지털혁신지원실', 'stack': ['기술 스택 없음']}
{'title': '데이터플랫폼 프론트엔드 개발자', 'company_name': '카카오헬스케어', 'detail_url': 'https://www.wanted.co.kr/wd/245520', 'platform_name': 'wanted', 'end_date': '2024.11.04', 'career': '경력 3년 이상', 'region': '경기도 성남시 분당 구 판교역로 166, 카카오판교아지트 B동 5층', 'stack': ['기술 스택 없음']}
{'title': '데이터 분석가(3년차)', 'company_name': '단색', 'detail_url': 'https://www.wanted.co.kr/wd/119414', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-4년', 'region': '서울시 마포구 월드컵북로 4길 51', 'stack': ['데이터 분석']}
{'title': '데이터 플랫폼 Back-End 개발/운영', 'company_name': '펑타이그레이터차이나(피티코리아)', 'detail_url': 'https://www.wanted.co.kr/wd/234032', 'platform_name': 'wanted', 'end_date': '2024.10.27', 'career': '경력 3-12년', 'region': '(06241) 서울 강남구 강남대로 372 FINE TOWER - 신분당선 강남 에서 200m 이내', 'stack': ['기술 스택 없음']}
{'title': 'Backend Engineer', 'company_name': '데이터라이즈', 'detail_url': 'https://www.wanted.co.kr/wd/81841', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 4-10년', 'region': '서울특별시 강남구 도곡로 205, YK빌딩 6층', 'stack': ['Git', 'Java', 'JavaScript', 'Python', 'AWS', 'Docker', 'FastAPI']}
{'title': '데이터 개발', 'company_name': '중앙일보', 'detail_url': 'https://www.wanted.co.kr/wd/147952', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 5-8년', 'region': '서울특별시 마포구 상암산로 48-6 중앙일보 빌딩', 'stack': ['Hadoop', 'Python', 'SQL', 'AWS', 'ElasticSearch', 'GCP']}
{'title': '데이터 분석 플랫폼 백엔드 개발', 'company_name': '엠트리센', 'detail_url': 'https://www.wanted.co.kr/wd/243476', 'platform_name': 'wanted', 'end_date': '2024.10.27', 'career': '경력 3-10년', 'region': '서울특별시 강남구 테헤란로 77길 13, 창조빌딩 10층', 'stack': ['Node.js']}
{'title': 'Frontend Engineer', 'company_name': '데이터라이즈', 'detail_url': 'https://www.wanted.co.kr/wd/233910', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '서울특별시 강남구 도곡로 205, YK빌딩 6층', 'stack': ['React', 'TypeScript', 'ES6', 'Redux', 'Angular']}
{'title': '[품질개선파트] QA Engineer', 'company_name': '한국신용데이터(KCD)', 'detail_url': 'https://www.wanted.co.kr/wd/243468', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2-5년', 'region': '서울특별시 강남구 테헤 란로 127, 5층(하나금융그룹 강남사옥)', 'stack': ['기술 스택 없음']}
{'title': '미디어데이터팀 수집 시스템 개발/관리/운영', 'company_name': '비큐에이아이', 'detail_url': 'https://www.wanted.co.kr/wd/242779', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 5년', 'region': '중구 퇴계로 385 (흥인동) 준타워 5,6,9층', 'stack': ['기술 스택 없음']}
교로 670, 유스페이스2 B동 403호', 'stack': ['Python', 'R', 'SQL', 'Tableau', 'NumPy']}
{'title': '[내집스캔] 개발 팀원 (3년 이상)', 'company_name': '한국부동산데이터연구소', 'detail_url': 'https://www.wanted.co.kr/wd/223783', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-15년', 'region': '서 울특별사 마포구 마포대로 122, 17층', 'stack': ['기술 스택 없음']}
{'title': '[Mnet Plus] Data Analyst', 'company_name': '씨제이이엔엠(CJ ENM)', 'detail_url': 'https://www.wanted.co.kr/wd/243652', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3년 이상', 'region': '서울시 마 포구 상암산로66 CJ ENM센터', 'stack': ['기술 스택 없음']}
{'title': 'Data Engineer', 'company_name': '브이원씨', 'detail_url': 'https://www.wanted.co.kr/wd/220548', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 3-10년', 'region': '강남구 삼성로 547 2층', 'stack': ['Django', 'MySQL', 'Python', 'SQL', 'AWS', 'GCP']}
{'title': '[데이터사업부]모니터링 솔루션 개발_React(프론트엔드)', 'company_name': '위즈코어', 'detail_url': 'https://www.wanted.co.kr/wd/239653', 'platform_name': 'wanted', 'end_date': '2024.10.26', 'career': '경력 3-20년', 'region': '서울시 성동구 아차산로17길 49 성수생각공장 데시앙플렉스 1505-8호', 'stack': ['React', 'JavaScript', 'HTML5', 'CSS3', 'Next.js']}
{'title': '플랫폼 서비스 기획자', 'company_name': '헥토데이터', 'detail_url': 'https://www.wanted.co.kr/wd/242935', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 6년 이상', 'region': '서울특별시 강남구 테헤란로 223, 큰길타워 5층', 'stack': ['Asana', 'Slack', 'Figma', 'Notion']}
{'title': '[삼쩜삼] Data Engineer', 'company_name': '자비스앤빌런즈(삼쩜삼)', 'detail_url': 'https://www.wanted.co.kr/wd/224022', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 4-20년', 'region': '서울 강남구 테헤란로44길 8, 아이콘역삼 9,10층 자비스앤빌런즈', 'stack': ['기술 스택 없음']}
{'title': '[NHN DATA] Social Biz(SaaS 솔루션) 세일즈', 'company_name': '엔에이치엔데이터(NHN DATA)', 'detail_url': 'https://www.wanted.co.kr/wd/242962', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '경력 2년 이상', 'region': '경기도 성남시 분당구 대왕판교로645번길', 'stack': ['기술 스택 없음']}
{'title': 'Data Scientist', 'company_name': '애자일소다', 'detail_url': 'https://www.wanted.co.kr/wd/242483', 'platform_name': 'wanted', 'end_date': '상시채용', 'career': '신입-경력 20년', 'region': '서울시 강남구 선릉로 525 인포스톰빌딩 3층', 'stack': ['기술 스택 없음']}